<a href="https://colab.research.google.com/github/Gabriel-Marinho-Godinho/TCC_Gabriel_Marinho_Godinho/blob/main/TCC_Gabriel_Marinho_Godinho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Análise Exploratória das Tarifas Aéreas praticadas no Brasil (2019 a 2022)
---

### ANAC - Estatísticas - Organização e Tratamento dos Dados


Versão 30/11/2023 ✈

## Introdução


Este notebook consolida o trabalho de **organização** e **tratamento** dos dados coletados nas bases da ANAC, no intervalo entre os anos de 2019 e 2022. Inicialmente, foi necessário estudar os bancos de dados para extrair os dados desejados para a nossa pesquisa e fazer as devidas correlações entre eles. Além disso, alguns dados serviram como insumo inicial para o cálculo de outras variáveis, como será apresentado a seguir no *notebook*.

Os bancos de dados que originalmente serão organizados e tratados neste notebook são:
* Estatísticas Básicas e Combinadas (2019 - 2022)
  
  * **Fonte**: AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL - ANAC. Microdados. Disponível em: <https://www.gov.br/anac/pt-br/assuntos/regulados/empresas-aereas/Instrucoes-para-a-elaboracao-e-apresentacao-das-demonstracoes-contabeis/envio-de-informacoes/microdados>. Acesso em: 25 nov. 2023.

* Lista de Aeródromos Públicos V2

  * **Fonte**: AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL - ANAC. Metadados do conjunto de dados: Lista de aeródromos públicos V2. Disponível em: <https://www.anac.gov.br/acesso-a-informacao/dados-abertos/areas-de-atuacao/aerodromos/lista-de-aerodromos-publicos-v2/70-lista-de-aerodromos-publicos-v2>. Acesso em: 25 nov. 2023

---
## **Objetivo Geral da Pesquisa**
**O objetivo deste trabalho é analisar e compreender as variáveis apresentadas pelo banco de dados da ANAC que influenciaram as tarifas dos voos tradicionais entre 2019 e 2022, e correlacionar essas percepções aos serviços de transporte por eVTOL no modelo de Malha Regional Aérea (RAM) brasileira.**

---

## Inicialização - Instalação de Pacotes Python

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/MyDrive/Colab Notebooks/ANAC

## **Passo 1** - Inicialização - Aeródromos e Distâncias entre Aeroportos

**Descrição:** leitura, organização e transformação dos arquivos de aeródromos da ANAC e criação do arquivo de distâncias, originado do cálculo das distâncias entre as combinações dos aeroportos de origem e destino.


### Etapa I. Acessando 'Lista de Aeródromos Públicos V2

Construção inicial do arquivo salvo em ./Dados/Aerodromos/aeroportos2.csv

**Descrição:** Nesta etapa foi realizada a leitura do arquivo da ANAC, organização e transformação do arquivo, excluindo as variáveis que não são de interesse para a pesquisa.

In [ ]:
# Leitura do arquivo:
aeroportos = pd.read_csv('./Dados/Aerodromos/AerodromosPublicos.csv',
                        encoding='cp1252', delimiter = ';',skiprows=1)

In [ ]:
# Tamanho do arquivo:
aeroportos.shape

In [ ]:
# Visualização do arquivo:
aeroportos.head()

In [ ]:
# Variáveis do arquivo:
aeroportos.columns

In [ ]:
# Exclusão das variáveis que não são de interesse para a pesquisa:
aer_coord = aeroportos.drop(columns=['CIAD', 'Nome', 'Município', 'UF',
      'Latitude', 'Longitude','Altitude', 'Operação Diurna',
      'Operação Noturna', 'Designação 1','Comprimento 1', 'Largura 1',
      'Resistência 1', 'Superfície 1','Designação 2', 'Comprimento 2',
      'Largura 2', 'Resistência 2','Superfície 2', 'Situação',
      'Validade do Registro','Portaria de Registro', 'Link Portaria'])

In [ ]:
# Renomeando as variáveis do arquivo:
aer_coord.columns = ['OACI','Municipio','UF','Lat','Long']

In [ ]:
# Eliminando os dados vazios (NA):
aer_coord = aer_coord.dropna()

In [ ]:
# Salvando o arquivo "aeroportos2.csv":
aer_coord.to_csv('./Dados/Aerodromos/aeroportos2.csv',index=False)

### Etapa II. Definindo a Função para o Cálculo da Distância entre Aeródromos
**Descrição:** Esta etapa foi realizada com base no projeto 'airportsdata20231017', que faz o levantamento da latitude e longitude para um conjunto de aeroportos. Através da latitude e longitude de um aeroporto de origem para um aeroporto de destino, foi possível calacular a distância entre eles.

**Fonte:** BORSETTI, M. Airportsdata 20231017. 2022. Disponível em: <https://pypi.org/project/airportsdata/>. Acesso em: 26 nov. 2023.

In [ ]:
# Instalando Airportsdata:
%pip install -U airportsdata

In [ ]:
import airportsdata

In [ ]:
# Buscando por código ICAO:
airports = airportsdata.load()  # key is ICAO code (default)
print(airports['SBBV'])

In [ ]:
#Buscando por código IATA
airports = airportsdata.load('IATA')  # key is IATA code, not the default
print(airports['BSB'])

In [ ]:
# Função para calcular distância de grande círculo entre aeroportos:
  # Observação: Distância de grande círculo é a menor distância entre dois pontos em uma esfera, medida ao longo da superfície da esfera.

import math

def distance_on_unit_sphere(lat1, long1, lat2, long2):

    # Converta a latitude e a longitude em coordenadas esféricas em radianos:
    degrees_to_radians = math.pi/180.0

    # phi = 90 - latitude:
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians

    # theta = longitude:
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians

    # Calcular a distância esférica a partir de coordenadas esféricas

    # Para dois locais em coordenadas esféricas
    # (1, theta, phi) and (1, theta, phi)
    # cosine( arc length ) =
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length

    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) +
           math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )

    # Multiplique o arco pelo raio da Terra em seu conjunto de unidades desejado (nesse caso, KM) para obter o comprimento.
    arc = arc * 6378.14 #raio da Terra em Quilômetros

    return arc

### Etapa III. Criando arquivo com distância entre aeroportos

Construção inicial do arquivo salvo em ./Dados/Tarifas/distancias.csv

**Descrição:** Esta etapa foi realizada para criar o arquivo de distâncias entre os aeroportos de origem e destino. Para isso, foram feitas todas as combinações possíveis entre aeropotos de origem e destino e, com base na latitude e longitude de cada um, foi aplicada a função *distance_on_unit_sphere* para calcular a distância entre eles (KM).

In [ ]:
# Leitura do arquivo:
aerodromo = pd.read_csv('./Dados/Aerodromos/aeroportos2.csv')

In [ ]:
# Visualização do arquivo:
aerodromo.head()

In [ ]:
# Criando uma lista com os dados da variável OACI (códigos dos aeroportos):
aerod = aerodromo.OACI.tolist()

In [ ]:
# 1. Criando as combinações possíveis de aeroportos (OACIs) de origem e destino e aplicando a função 'distance_on_unit_sphere'para as suas respectivas latitudes e longitudes de origem e destino.
# 2. Append de 'i' na lista orig_aer, de 'j' na lista dest_aer e de 'distancia' na lista dist_aer:
orig_aer = []
dest_aer = []
dist_aer = []
for i in aerod:
  for j in aerod:
    if i != j:
      o = aerodromo[aerodromo.OACI == i]
      d = aerodromo[aerodromo.OACI == j]
      lat1 = o['Lat'].values[0]
      lon1 = o['Long'].values[0]
      lat2 = d['Lat'].values[0]
      lon2 = d['Long'].values[0]
      distancia = distance_on_unit_sphere(lat1, lon1, lat2, lon2)
    else:
      distancia = 0.0
    orig_aer.append(i)
    dest_aer.append(j)
    dist_aer.append(distancia)

In [ ]:
# Criando o dataframe distancias:
orig_series = pd.Series(orig_aer)
dest_series = pd.Series(dest_aer)
dist_series = pd.Series(dist_aer)
frame2 = {'orig': orig_series,
          'dest': dest_series,
          'dist': dist_series}
distancias = pd.DataFrame(frame2)

In [ ]:
# Visualizando o dataframe distancias:
distancias.head()

In [ ]:
# Salvando o arquivo "distancias.csv":
distancias.to_csv('./Dados/Tarifas/distancias.csv',index=False)

### Etapa IV. Leitura do arquivo "distancias.csv" e transformação inicial para float

Salvo em ./Dados/Tarifas/distancias2.csv

**Descrição:** Esta etapa foi realizada para transformar o tipo do dado da variável distância de *string* para *float*.

In [ ]:
# Leitura do arquivo:
dados_distancias = pd.read_csv('./Dados/Tarifas/distancias.csv')

In [ ]:
# Visualização do arquivo:
dados_distancias.head()

In [ ]:
# Substituindo ',' por '.':
dados_distancias.dist = dados_distancias.dist.str.replace(',','.')

In [ ]:
# Transformando tipo de dado para 'float' (número real):
dados_distancias.dist = dados_distancias.dist.astype(float)

In [ ]:
# Salvando o arquivo "distancias2.csv":
dados_distancias.to_csv('./Dados/Aerodromos/distancias2.csv',index=False)

## **Passo 2** - Tarifas - Transformação de Dados

**Descrição:** leitura, organização e transformação dos arquivos de tarifas disponibilizados pela ANAC, entre 2019 e 2022.

**Descrição das Variáveis:** ANAC. Microdados de tarifas aéreas comercializadas. Disponível em: <https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/microdados-de-tarifas-aereas-comercializadas>. Acesso em: 26 nov. 2023.

**Atenção:** os arquivos baixados são .CSV, mas no formato Microsoft Excel, *comma separated values*, portanto, não tem o encoding utf-8. Além disso, o Excel em português salva o CSV separado com ponto e vírgula (';').

### Etapa I. Tarifas - Transformação de Dados 2019

#### Etapa I.I. Leitura e agrupamento de arquivos de Tarifas 2019

Salvo em ./Dados/Tarifas/tarifas_2019.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de tarifas disponibilizados pela ANAC para 2019.

In [ ]:
# Leitura do arquivo:
files = ['201901.CSV', '201902.CSV','201903.CSV','201904.CSV',
         '201905.CSV','201906.CSV','201907.CSV','201908.CSV',
         '201909.CSV','201910.CSV','201911.CSV','201912.CSV']

In [ ]:
# Estruturando o dataframe:
df = pd.DataFrame(columns=['ANO','MES','EMPRESA','ORIGEM',
                           'DESTINO','TARIFA','ASSENTOS'])

In [ ]:
# Agrupando os arquivos no dataframe estruturado:
for file in files:
    data = pd.read_csv(f'./Dados/Tarifas/Tarifas_2019/{file}',
                       encoding='cp1252', delimiter = ';')
    data.columns = ['ANO','MES','EMPRESA','ORIGEM',
                    'DESTINO','TARIFA','ASSENTOS']
    df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualização do dataframe:
df.head()

In [ ]:
# Salvando o arquivo "tarifas_2019.csv":
df.to_csv('./Dados/Tarifas/tarifas_2019.csv', index=False)

#### Etapa I.II. Obtenção das Tarifas em 2019 e transformação inicial

Salvos em ./Dados/Tarifas/tarifas9.csv

**Descrição:** Nesta etapa foi realizada a transformação dos dados do arquivo tarifas_2019, o adequando para - posteriormente - ser analisado.

In [ ]:
# Leitura do arquivo:
dados_tarifas = pd.read_csv('./Dados/Tarifas/tarifas_2019.csv')

In [ ]:
# Tamanho do arquivo:
dados_tarifas.shape

In [ ]:
# Visualização do arquivo:
dados_tarifas.head()

In [ ]:
# Padronização dos dados respecivos à variável MES:
dados_tarifas.MES.unique()

In [ ]:
# Substituindo ',' por '.' na variável TARIFA:
dados_tarifas.TARIFA = dados_tarifas.TARIFA.str.replace(',','.')

In [ ]:
# Alterando o tipo de dado da variável TARIFA para float:
dados_tarifas.TARIFA = dados_tarifas.TARIFA.astype(float)

In [ ]:
# Salvando o arquivo "tarifas9.csv":
dados_tarifas.to_csv('./Dados/Tarifas/tarifas9.csv',index=False)

### Etapa II. Tarifas - Transformação de Dados 2020

#### Etapa II.I. Leitura e agrupamento de arquivos de Tarifas 2020

Salvo em ./Dados/Tarifas/tarifas_2020.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de tarifas disponibilizados pela ANAC para 2020.

In [ ]:
# Leitura do arquivo:
files = ['202001.CSV', '202002.CSV','202003.CSV','202004.CSV',
         '202005.CSV','202006.CSV','202007.CSV','202008.CSV',
         '202009.CSV','202010.CSV','202011.CSV','202012.CSV']

In [ ]:
# Estruturando o dataframe:
df = pd.DataFrame(columns=['ANO','MES','EMPRESA','ORIGEM',
                           'DESTINO','TARIFA','ASSENTOS'])

In [ ]:
# Agrupando os arquivos no dataframe estruturado:
for file in files:
    data = pd.read_csv(f'./Dados/Tarifas/Tarifas_2020/{file}',
                       encoding='cp1252', delimiter = ';')
    data.columns = ['ANO','MES','EMPRESA','ORIGEM',
                    'DESTINO','TARIFA','ASSENTOS']
    df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualização do dataframe:
df.head()

In [ ]:
# Salvando o arquivo "tarifas_2020.csv":
df.to_csv('./Dados/Tarifas/tarifas_2020.csv', index=False)

#### Etapa II.II. Obtenção das Tarifas em 2020 e transformação inicial

Salvos em ./Dados/Tarifas/tarifas0.csv

**Descrição:** Nesta etapa foi realizada a transformação dos dados do arquivo tarifas_2020, o adequando para - posteriormente - ser analisado.

In [ ]:
# Leitura do arquivo:
dados_tarifas = pd.read_csv('./Dados/Tarifas/tarifas_2020.csv')

In [ ]:
# Tamanho do arquivo:
dados_tarifas.shape

In [ ]:
# Visualização do arquivo:
dados_tarifas.head()

In [ ]:
# Padronização dos dados respecivos à variável MES:
dados_tarifas.MES.unique()

In [ ]:
# Substituindo ',' por '.' na variável TARIFA:
dados_tarifas.TARIFA = dados_tarifas.TARIFA.str.replace(',','.')

In [ ]:
# Alterando o tipo de dado da variável TARIFA para float:
dados_tarifas.TARIFA = dados_tarifas.TARIFA.astype(float)

In [ ]:
# Salvando o arquivo "tarifas0.csv":
dados_tarifas.to_csv('./Dados/Tarifas/tarifas0.csv',index=False)

### Etapa III. Tarifas - Transformação de Dados 2021

#### Etapa III.I. Leitura e agrupamento de arquivos de Tarifas 2021

Salvo em ./Dados/Tarifas/tarifas_2021.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de tarifas disponibilizados pela ANAC para 2021.

In [ ]:
# Leitura do arquivo:
files = ['202101.CSV', '202102.CSV','202103.CSV','202104.CSV',
         '202105.CSV','202106.CSV','202107.CSV','202108.CSV',
         '202109.CSV','202110.CSV','202111.CSV','202112.CSV']

In [ ]:
# Estruturando o dataframe:
df = pd.DataFrame(columns=['ANO','MES','EMPRESA','ORIGEM',
                           'DESTINO','TARIFA','ASSENTOS'])

In [ ]:
# Agrupando os arquivos no dataframe estruturado:
for file in files:
    data = pd.read_csv(f'./Dados/Tarifas/Tarifas_2021/{file}',
                       encoding='cp1252', delimiter = ';')
    if file == '202112.CSV':
        data = data.iloc[:,1:]
    data.columns = ['ANO','MES','EMPRESA','ORIGEM',
                    'DESTINO','TARIFA','ASSENTOS']
    df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualização do dataframe:
df.head()

In [ ]:
# Salvando o arquivo "tarifas_2021.csv":
df.to_csv('./Dados/Tarifas/tarifas_2021.csv', index=False)

#### Etapa III.II. Obtenção das Tarifas em 2021 e transformação inicial

Salvos em ./Dados/Tarifas/tarifas1.csv

**Descrição:** Nesta etapa foi realizada a transformação dos dados do arquivo tarifas_2021, o adequando para - posteriormente - ser analisado.

In [ ]:
# Leitura do arquivo:
dados_tarifas = pd.read_csv('./Dados/Tarifas/tarifas_2021.csv')

In [ ]:
# Tamanho do arquivo:
dados_tarifas.shape

In [ ]:
# Visualização do arquivo:
dados_tarifas.head()

In [ ]:
# Padronização dos dados respecivos à variável MES:
dados_tarifas.MES.unique()

In [ ]:
# Substituindo ',' por '.' na variável TARIFA:
dados_tarifas.TARIFA = dados_tarifas.TARIFA.str.replace(',','.')

In [ ]:
# Alterando o tipo de dado da variável TARIFA para float:
dados_tarifas.TARIFA = dados_tarifas.TARIFA.astype(float)

In [ ]:
# Salvando o arquivo "tarifas1.csv":
dados_tarifas.to_csv('./Dados/Tarifas/tarifas1.csv',index=False)

### Etapa IV. Tarifas - Transformação de Dados 2022

#### Etapa IV.I. Leitura e agrupamento de arquivos de Tarifas 2022

Salvo em ./Dados/Tarifas/tarifas_2022.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de tarifas disponibilizados pela ANAC para 2022.

In [ ]:
# Leitura do arquivo:
files = ['202201.CSV', '202202.CSV','202203.CSV','202204.CSV',
         '202205.CSV','202206.CSV','202207.CSV','202208.CSV',
         '202209.CSV','202210.CSV','202211.CSV','202212.CSV']

In [ ]:
# Estruturando o dataframe:
df = pd.DataFrame(columns=['ANO','MES','EMPRESA','ORIGEM',
                           'DESTINO','TARIFA','ASSENTOS'])

In [ ]:
# Agrupando os arquivos no dataframe estruturado:
for file in files:
    data = pd.read_csv(f'./Dados/Tarifas/Tarifas_2022/{file}',
                       encoding='cp1252', delimiter = ';')
    if file == '202204.CSV':
        data = data.iloc[:,1:]
    data.columns = ['ANO','MES','EMPRESA','ORIGEM',
                    'DESTINO','TARIFA','ASSENTOS']
    df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualização do dataframe:
df.head()

In [ ]:
# Salvando o arquivo "tarifas_2022.csv":
df.to_csv('./Dados/Tarifas/tarifas_2022.csv', index=False)

#### Etapa IV.II. Obtenção das Tarifas em 2022 e transformação inicial

Salvos em ./Dados/Tarifas/tarifas2.csv

**Descrição:** Nesta etapa foi realizada a transformação dos dados do arquivo tarifas_2022, o adequando para - posteriormente - ser analisado.

In [ ]:
# Leitura do arquivo:
dados_tarifas = pd.read_csv('./Dados/Tarifas/tarifas_2022.csv')

In [ ]:
# Tamanho do arquivo:
dados_tarifas.shape

In [ ]:
# Visualização do arquivo:
dados_tarifas.head()

In [ ]:
# Padronização dos dados respecivos à variável MES:
dados_tarifas.MES.unique()

In [ ]:
# Substituindo ',' por '.' na variável TARIFA:
dados_tarifas.TARIFA = dados_tarifas.TARIFA.str.replace(',','.')

In [ ]:
# Alterando o tipo de dado da variável TARIFA para float:
dados_tarifas.TARIFA = dados_tarifas.TARIFA.astype(float)

In [ ]:
# Salvando o arquivo "tarifas2.csv":
dados_tarifas.to_csv('./Dados/Tarifas/tarifas2.csv',index=False)

## **Passo 3** - Demanda e Oferta - Transformação de Dados

**Descrição:** leitura, organização e transformação dos arquivos de estatisticas disponibilizados pela ANAC, respectivos aos voos realizados entre 2019 e 2022.

**Descrição das Variáveis:** ANAC. Descrição das variáveis (microdados). Disponível em: <https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/mercado-do-transporte-aereo/demanda-e-oferta/descricao-das-variaveis-microdados>. Acesso em: 26 nov. 2023.

**Atenção:** os arquivos baixados são .CSV, mas no formato Microsoft Excel, *comma separated values*, portanto, não tem o encoding utf-8. Além disso, o Excel em português salva o CSV separado com ponto e vírgula (';').

### Etapa I. Demanda e Oferta - Básica

**Etapa Básica (flight stage):** As etapas básicas são aquelas realizadas pela aeronave desde a sua decolagem até o próximo pouso, independentemente de onde tenha ocorrido o embarque ou o desembarque do objeto de transporte. Os dados estatísticos das etapas básicas representam o status da aeronave em cada etapa do voo, apresentando a movimentação de cargas e passageiros entre os aeródromos de origem e destino da aeronave. É a operação de uma aeronave entre uma decolagem e o próximo pouso, a ligação entre dois aeródromos.

**Fonte:** ANAC. Descrição das variáveis (microdados). Disponível em: <https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/mercado-do-transporte-aereo/demanda-e-oferta/descricao-das-variaveis-microdados>. Acesso em: 26 nov. 2023

#### Etapa I.I. Leitura e agrupamento de arquivos de estatísticas básicas 2019

Salvo em ./Dados/DemandaeOferta/estatisticas_2019.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de estatisticas disponibilizados pela ANAC para 2019.

In [ ]:
# Leitura do arquivo:
files = ['basica2019-01.zip', 'basica2019-02.zip','basica2019-03.zip',
         'basica2019-04.zip', 'basica2019-05.zip','basica2019-06.zip',
         'basica2019-07.zip', 'basica2019-08.zip','basica2019-09.zip',
         'basica2019-10.zip', 'basica2019-11.zip','basica2019-12.zip']

In [ ]:
# Criação do dataframe:
df = pd.DataFrame()

In [ ]:
# Concatenação dos arquivos no dataframe:
for file in files:
  print(file)
  data = pd.read_csv(f'./Dados/DemandaeOferta/2019_basica/{file}',
                     encoding='cp1252', delimiter = ';',low_memory=False)
  df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualização do dataframe:
df.head()

In [ ]:
# Salvando o arquivo "estatisticas_2019.csv":
df.to_csv('./Dados/DemandaeOferta/estatisticas_2019.csv', index=False)

#### Etapa I.II. Obtenção das estatísticas básicas de Demanda e Oferta em 2019

Salvo em /Dados/Estatisticas/estatisticas9.csv

**Descrição:** Nesta etapa foi realizada a transformação do arquivo estatisticas_2019.

In [ ]:
# Leitura do arquivo:
dados_estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas_2019.csv',
                                 low_memory=False)

In [ ]:
# Dimensões do arquivo:
dados_estatisticas.shape

In [ ]:
# Padronização da variável mês:
dados_estatisticas['nr_mes_referencia'].unique()

In [ ]:
dados_estatisticas.columns

In [ ]:
# Filtrando o dataframe com a natureza da etapa de voo = doméstica, serviço referente ao tipo de linha = passageiro e tipo de Voo = regular:
dados_estatisticas = dados_estatisticas[
    (dados_estatisticas['ds_natureza_etapa'] == 'DOMÉSTICA') & \
    (dados_estatisticas['ds_servico_tipo_linha'] == 'PASSAGEIRO') & \
    (dados_estatisticas['ds_grupo_di'] == 'REGULAR')]
# Com isso, excluem-se os dados que não são do interesse da pesquisa.

In [ ]:
dados_estatisticas.info()

In [ ]:
dados_estatisticas.isna().sum()

In [ ]:
# Exclusão das variáveis que não são do interesse para a pesquisa:
dados_estatisticas = dados_estatisticas.drop(columns=
    ['id_basica','id_empresa','sg_empresa_iata','nm_pais','ds_tipo_empresa',
     'nr_singular','id_di','cd_di','ds_di','ds_grupo_di','dt_referencia',
     'nr_ano_referencia','nr_semestre_referencia','nm_semestre_referencia',
     'nr_trimestre_referencia','nm_trimestre_referencia','nr_mes_referencia',
     'nm_mes_referencia','nr_semana_referencia','nm_dia_semana_referencia',
     'nr_dia_referencia','nr_ano_mes_referencia','id_tipo_linha',
     'cd_tipo_linha','ds_tipo_linha','ds_natureza_tipo_linha','ds_servico_tipo_linha',
     'ds_natureza_etapa','nr_ano_partida_real','nr_semestre_partida_real',
     'nm_semestre_partida_real','nr_trimestre_partida_real','nm_trimestre_partida_real',
     'nm_mes_partida_real','nr_ano_mes_partida_real','id_aerodromo_origem',
     'nm_aerodromo_origem','nm_municipio_origem','nm_pais_origem','nm_continente_origem',
     'nr_ano_chegada_real','nr_semestre_chegada_real','nm_semestre_chegada_real',
     'nr_trimestre_chegada_real','nm_trimestre_chegada_real','nm_mes_chegada_real',
     'nr_ano_mes_chegada_real','id_aerodromo_destino','nm_aerodromo_destino',
     'nm_municipio_destino','nm_regiao_destino','nm_pais_destino','nm_continente_destino',
     'nr_escala_destino','id_arquivo','nm_arquivo','nr_linha','dt_sistema']
)

In [ ]:
# Dimensão do dataframe:
dados_estatisticas.shape

In [ ]:
# Salvando o arquivo "estatisticas9.csv":
dados_estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas9.csv',
                          index=False)

#### Etapa I.III. Leitura e agrupamento de arquivos de estatísticas básicas 2020

Salvo em ./Dados/DemandaeOferta/estatisticas_2020.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de estatisticas disponibilizados pela ANAC para 2020.

In [ ]:
# Leitura do arquivo:
files = ['basica2020-01.zip', 'basica2020-02.zip','basica2020-03.zip',
         'basica2020-04.zip', 'basica2020-05.zip','basica2020-06.zip',
         'basica2020-07.zip', 'basica2020-08.zip','basica2020-09.zip',
         'basica2020-10.zip', 'basica2020-11.zip','basica2020-12.zip']

In [ ]:
# Criação do dataframe:
df = pd.DataFrame()

In [ ]:
# Concatenação dos arquivos no dataframe:
for file in files:
  print(file)
  data = pd.read_csv(f'./Dados/DemandaeOferta/2020_basica/{file}',
                     encoding='cp1252', delimiter = ';',low_memory=False)
  df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualização do dataframe:
df.head()

In [ ]:
# Salvando o arquivo "estatisticas_2020.csv":
df.to_csv('./Dados/DemandaeOferta/estatisticas_2020.csv', index=False)

#### Etapa I.IV. Obtenção das estatísticas básicas de Demanda e Oferta em 2020

Salvo em /Dados/Estatisticas/estatisticas0.csv

**Descrição:** Nesta etapa foi realizada a transformação do arquivo estatisticas_2020.

In [ ]:
# Leitura do arquivo:
dados_estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas_2020.csv',
                                 low_memory=False)

In [ ]:
# Dimensões do arquivo:
dados_estatisticas.shape

In [ ]:
# Padronização da variável mês:
dados_estatisticas['nr_mes_referencia'].unique()

In [ ]:
dados_estatisticas.columns

In [ ]:
# Filtrando o dataframe com a natureza da etapa de voo = doméstica, serviço referente ao tipo de linha = passageiro e tipo de Voo = regular:
dados_estatisticas = dados_estatisticas[
    (dados_estatisticas['ds_natureza_etapa'] == 'DOMÉSTICA') & \
    (dados_estatisticas['ds_servico_tipo_linha'] == 'PASSAGEIRO') & \
    (dados_estatisticas['ds_grupo_di'] == 'REGULAR')]

In [ ]:
dados_estatisticas.info()

In [ ]:
dados_estatisticas.isna().sum()

In [ ]:
# Exclusão das variáveis que não são do interesse para a pesquisa:
dados_estatisticas = dados_estatisticas.drop(columns=
    ['id_basica','id_empresa','sg_empresa_iata','nm_pais','ds_tipo_empresa',
     'nr_singular','id_di','cd_di','ds_di','ds_grupo_di','dt_referencia',
     'nr_ano_referencia','nr_semestre_referencia','nm_semestre_referencia',
     'nr_trimestre_referencia','nm_trimestre_referencia','nr_mes_referencia',
     'nm_mes_referencia','nr_semana_referencia','nm_dia_semana_referencia',
     'nr_dia_referencia','nr_ano_mes_referencia','id_tipo_linha',
     'cd_tipo_linha','ds_tipo_linha','ds_natureza_tipo_linha','ds_servico_tipo_linha',
     'ds_natureza_etapa','nr_ano_partida_real','nr_semestre_partida_real',
     'nm_semestre_partida_real','nr_trimestre_partida_real','nm_trimestre_partida_real',
     'nm_mes_partida_real','nr_ano_mes_partida_real','id_aerodromo_origem',
     'nm_aerodromo_origem','nm_municipio_origem','nm_pais_origem','nm_continente_origem',
     'nr_ano_chegada_real','nr_semestre_chegada_real','nm_semestre_chegada_real',
     'nr_trimestre_chegada_real','nm_trimestre_chegada_real','nm_mes_chegada_real',
     'nr_ano_mes_chegada_real','id_aerodromo_destino','nm_aerodromo_destino',
     'nm_municipio_destino','nm_regiao_destino','nm_pais_destino','nm_continente_destino',
     'nr_escala_destino','id_arquivo','nm_arquivo','nr_linha','dt_sistema']
)

In [ ]:
# Dimensão do dataframe:
dados_estatisticas.shape

In [ ]:
# Salvando o arquivo "estatisticas0.csv":
dados_estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas0.csv',
                          index=False)

#### Etapa I.V. Leitura e agrupamento de arquivos de estatísticas básicas 2021

Salvo em ./Dados/DemandaeOferta/estatisticas_2021.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de estatisticas disponibilizados pela ANAC para 2021.

In [ ]:
# Leitura do arquivo:
files = ['basica2021-01.zip', 'basica2021-02.zip','basica2021-03.zip',
         'basica2021-04.zip', 'basica2021-05.zip','basica2021-06.zip',
         'basica2021-07.zip', 'basica2021-08.zip','basica2021-09.zip',
         'basica2021-10.zip', 'basica2021-11.zip','basica2021-12.zip']

In [ ]:
# Criação do dataframe:
df = pd.DataFrame()

In [ ]:
# Concatenação dos arquivos no dataframe:
for file in files:
  print(file)
  data = pd.read_csv(f'./Dados/DemandaeOferta/2021_basica/{file}',
                     encoding='cp1252', delimiter = ';',low_memory=False)
  df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualização do dataframe:
df.head()

In [ ]:
# Salvando o arquivo "estatisticas_2021.csv"
df.to_csv('./Dados/DemandaeOferta/estatisticas_2021.csv', index=False)

#### Etapa I.VI. Obtenção das estatísticas básicas de Demanda e Oferta em 2021

Salvo em /Dados/Estatisticas/estatisticas1.csv

**Descrição:** Nesta etapa foi realizada a transformação do arquivo estatisticas_2021.

In [ ]:
# Leitura do arquivo:
dados_estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas_2021.csv',
                                 low_memory=False)

In [ ]:
# Dimensões do arquivo:
dados_estatisticas.shape

In [ ]:
# Padronização da variável mês:
dados_estatisticas['nr_mes_referencia'].unique()

In [ ]:
dados_estatisticas.columns

In [ ]:
# Filtrando o dataframe com a natureza da etapa de voo = doméstica, serviço referente ao tipo de linha = passageiro e tipo de Voo = regular:
dados_estatisticas = dados_estatisticas[
    (dados_estatisticas['ds_natureza_etapa'] == 'DOMÉSTICA') & \
    (dados_estatisticas['ds_servico_tipo_linha'] == 'PASSAGEIRO') & \
    (dados_estatisticas['ds_grupo_di'] == 'REGULAR')]

In [ ]:
dados_estatisticas.info()

In [ ]:
dados_estatisticas.isna().sum()

In [ ]:
# Exclusão das variáveis que não são do interesse para a pesquisa:
dados_estatisticas = dados_estatisticas.drop(columns=
    ['id_basica','id_empresa','sg_empresa_iata','nm_pais','ds_tipo_empresa',
     'nr_singular','id_di','cd_di','ds_di','ds_grupo_di','dt_referencia',
     'nr_ano_referencia','nr_semestre_referencia','nm_semestre_referencia',
     'nr_trimestre_referencia','nm_trimestre_referencia','nr_mes_referencia',
     'nm_mes_referencia','nr_semana_referencia','nm_dia_semana_referencia',
     'nr_dia_referencia','nr_ano_mes_referencia','id_tipo_linha',
     'cd_tipo_linha','ds_tipo_linha','ds_natureza_tipo_linha','ds_servico_tipo_linha',
     'ds_natureza_etapa','nr_ano_partida_real','nr_semestre_partida_real',
     'nm_semestre_partida_real','nr_trimestre_partida_real','nm_trimestre_partida_real',
     'nm_mes_partida_real','nr_ano_mes_partida_real','id_aerodromo_origem',
     'nm_aerodromo_origem','nm_municipio_origem','nm_pais_origem','nm_continente_origem',
     'nr_ano_chegada_real','nr_semestre_chegada_real','nm_semestre_chegada_real',
     'nr_trimestre_chegada_real','nm_trimestre_chegada_real','nm_mes_chegada_real',
     'nr_ano_mes_chegada_real','id_aerodromo_destino','nm_aerodromo_destino',
     'nm_municipio_destino','nm_regiao_destino','nm_pais_destino','nm_continente_destino',
     'nr_escala_destino','id_arquivo','nm_arquivo','nr_linha','dt_sistema']
)

In [ ]:
# Dimensão do dataframe:
dados_estatisticas.shape

In [ ]:
# Salvando o arquivo "estatisticas1.csv":
dados_estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas1.csv',
                          index=False)

#### Etapa I.VII. Leitura e agrupamento de arquivos de estatísticas básicas 2022

Salvo em ./Dados/DemandaeOferta/estatisticas_2022.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de estatisticas disponibilizados pela ANAC para 2021.

In [ ]:
# Leitura do arquivo:
files = ['basica2022-01.zip', 'basica2022-02.zip','basica2022-03.zip',
         'basica2022-04.zip', 'basica2022-05.zip','basica2022-06.zip',
         'basica2022-07.zip', 'basica2022-08.zip','basica2022-09.zip',
         'basica2022-10.zip', 'basica2022-11.zip','basica2022-12.zip']

In [ ]:
# Criação do dataframe:
df = pd.DataFrame()

In [ ]:
# Concatenação dos arquivos no dataframe:
for file in files:
  print(file)
  data = pd.read_csv(f'./Dados/DemandaeOferta/2022_basica/{file}',
                     encoding='cp1252', delimiter = ';',low_memory=False)
  df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualização do dataframe:
df.head()

In [ ]:
# Salvando o arquivo "estatisticas_2022.csv":
df.to_csv('./Dados/Estatisticas/estatisticas_2022.csv', index=False)

#### Etapa I.VIII. Obtenção das estatísticas básicas de Demanda e Oferta em 2022

Salvo em /Dados/Estatisticas/estatisticas2.csv

**Descrição:** Nesta etapa foi realizada a transformação do arquivo estatisticas_2022.

In [ ]:
 # Leitura do arquivo:
 dados_estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas_2022.csv',
                                 low_memory=False)

In [ ]:
# Dimensões do arquivo:
dados_estatisticas.shape

In [ ]:
# Padronização da variável mês:
dados_estatisticas['nr_mes_referencia'].unique()

In [ ]:
dados_estatisticas.columns

In [ ]:
# Filtrando o dataframe com a natureza da etapa de voo = doméstica, serviço referente ao tipo de linha = passageiro e tipo de Voo = regular:
dados_estatisticas = dados_estatisticas[
    (dados_estatisticas['ds_natureza_etapa'] == 'DOMÉSTICA') & \
    (dados_estatisticas['ds_servico_tipo_linha'] == 'PASSAGEIRO') & \
    (dados_estatisticas['ds_grupo_di'] == 'REGULAR')]

In [ ]:
dados_estatisticas.info()

In [ ]:
dados_estatisticas.isna().sum()

In [ ]:
# Exclusão das variáveis que não são do interesse para a pesquisa:
dados_estatisticas = dados_estatisticas.drop(columns=
    ['id_basica','id_empresa','sg_empresa_iata','nm_pais','ds_tipo_empresa',
     'nr_singular','id_di','cd_di','ds_di','ds_grupo_di','dt_referencia',
     'nr_ano_referencia','nr_semestre_referencia','nm_semestre_referencia',
     'nr_trimestre_referencia','nm_trimestre_referencia','nr_mes_referencia',
     'nm_mes_referencia','nr_semana_referencia','nm_dia_semana_referencia',
     'nr_dia_referencia','nr_ano_mes_referencia','id_tipo_linha',
     'cd_tipo_linha','ds_tipo_linha','ds_natureza_tipo_linha','ds_servico_tipo_linha',
     'ds_natureza_etapa','nr_ano_partida_real','nr_semestre_partida_real',
     'nm_semestre_partida_real','nr_trimestre_partida_real','nm_trimestre_partida_real',
     'nm_mes_partida_real','nr_ano_mes_partida_real','id_aerodromo_origem',
     'nm_aerodromo_origem','nm_municipio_origem','nm_pais_origem','nm_continente_origem',
     'nr_ano_chegada_real','nr_semestre_chegada_real','nm_semestre_chegada_real',
     'nr_trimestre_chegada_real','nm_trimestre_chegada_real','nm_mes_chegada_real',
     'nr_ano_mes_chegada_real','id_aerodromo_destino','nm_aerodromo_destino',
     'nm_municipio_destino','nm_regiao_destino','nm_pais_destino','nm_continente_destino',
     'nr_escala_destino','id_arquivo','nm_arquivo','nr_linha','dt_sistema']
)

In [ ]:
# Dimensão do dataframe:
dados_estatisticas.shape

In [ ]:
# Salvando o arquivo "estatisticas2.csv":
dados_estatisticas.to_csv('./Dados/Estatisticas/estatisticas2.csv',
                          index=False)

### Etapa II. Demanda e Oferta - Combinada

**Etapa Combinada (On flight origin and destination - OFOD):** As etapas combinadas identificam os pares de origem, onde ocorreu o embarque do objeto de transporte, e destino, onde ocorreu o desembarque do objeto de transporte, independentemente da existência de aeródromos intermediários, atendidos por determinado voo. É a etapa de voo vista com foco no objeto de transporte (pessoas e/ou cargas), com base no embarque e no desembarque nos aeródromos relacionados. Os dados estatísticos da etapa combinada informam a origem e destino dos passageiros e cargas transportadas no voo, independentemente das escalas realizadas.

**Fonte:** ANAC. Descrição das variáveis (microdados). Disponível em: <https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/mercado-do-transporte-aereo/demanda-e-oferta/descricao-das-variaveis-microdados>. Acesso em: 26 nov. 2023

#### Etapa II.I. Organização e Transformação dos Dados - Estatisticas Combinadas 2019

##### 1. Leitura e agrupamento de arquivos de estatísticas combinadas 2019

Salvo em ./Dados/DemandaeOferta/2019_combinada.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de estatisticas combinadas disponibilizados pela ANAC para 2019.

In [ ]:
# Leitura do arquivo:
files = ['combinada2019-01.zip', 'combinada2019-02.zip','combinada2019-03.zip',
         'combinada2019-04.zip', 'combinada2019-05.zip','combinada2019-06.zip',
         'combinada2019-07.zip', 'combinada2019-08.zip','combinada2019-09.zip',
         'combinada2019-10.zip', 'combinada2019-11.zip','combinada2019-12.zip']

In [ ]:
# Criando o dataframe:
df = pd.DataFrame()

In [ ]:
# Concatenando os arquivos no dataframe:
for file in files:
  print(file)
  data = pd.read_csv(f'./Dados/DemandaeOferta/2019_combinada/{file}',
                     encoding='cp1252', delimiter = ';',low_memory=False)
  df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualizando o dataframe:
df.head()

In [ ]:
# Salvando o arquivo "estatisticas_2019_combinadas.csv":
df.to_csv('./Dados/DemandaeOferta/estatisticas_2019_combinadas.csv', index=False)

##### 2. Obtenção das estatísticas combinadas de Demanda e Oferta em 2019

Salvo em /Dados/Estatisticas/estatisticas9c.csv

**Descrição:** Nesta etapa foi realizada a transformação do arquivo estatisticas_2019_combinadas.csv.

In [ ]:
# Leitura do arquivo:
dados_estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas_2019_combinadas.csv',
                                 low_memory=False)

In [ ]:
# Dimnesão do arquivo:
dados_estatisticas.shape

In [ ]:
# Padronizando os dados respectivos ao mês:
dados_estatisticas['nr_mes_referencia'].unique()

In [ ]:
dados_estatisticas.columns

In [ ]:
# Filtrando o dataframe com a natureza da etapa de voo = doméstica, serviço referente ao tipo de linha = passageiro e tipo de Voo = regular:
dados_estatisticas = dados_estatisticas[
    (dados_estatisticas['ds_natureza_etapa'] == 'DOMÉSTICA') & \
    (dados_estatisticas['ds_servico_tipo_linha'] == 'PASSAGEIRO') & \
    (dados_estatisticas['ds_grupo_di'] == 'REGULAR')]
# Com isso, foram excluídos os dados que não serão trabalhados nessa pesquisa.

In [ ]:
dados_estatisticas.info()

In [ ]:
dados_estatisticas.isna().sum()

In [ ]:
# Exclusão das variáveis que não serão estudadas na pesquisa:
dados_estatisticas = dados_estatisticas.drop(columns=
    ['id_combinada','id_empresa','sg_empresa_iata','nm_pais','ds_tipo_empresa',
     'nr_singular','id_di','cd_di','ds_di','ds_grupo_di','dt_referencia',
     'nr_ano_referencia','nr_semestre_referencia','nm_semestre_referencia',
     'nr_trimestre_referencia','nm_trimestre_referencia','nr_mes_referencia',
     'nm_mes_referencia','nr_semana_referencia','nm_dia_semana_referencia',
     'nr_dia_referencia','nr_ano_mes_referencia','id_tipo_linha',
     'cd_tipo_linha','ds_tipo_linha','ds_natureza_tipo_linha','ds_servico_tipo_linha',
     'ds_natureza_etapa','nr_ano_partida_real','nr_semestre_partida_real',
     'nm_semestre_partida_real','nr_trimestre_partida_real','nm_trimestre_partida_real',
     'nm_mes_partida_real','nr_ano_mes_partida_real','id_aerodromo_origem',
     'nm_aerodromo_origem','nm_municipio_origem','nm_pais_origem','nm_continente_origem',
     'nr_ano_chegada_real','nr_semestre_chegada_real','nm_semestre_chegada_real',
     'nr_trimestre_chegada_real','nm_trimestre_chegada_real','nm_mes_chegada_real',
     'nr_ano_mes_chegada_real','id_aerodromo_destino','nm_aerodromo_destino',
     'nm_municipio_destino','nm_regiao_destino','nm_pais_destino','nm_continente_destino',
     'nr_escala_destino','id_arquivo','nm_arquivo','nr_linha','dt_sistema']
)

In [ ]:
# Dimensão do dataframe:
dados_estatisticas.shape

In [ ]:
# Salvando o arquivo "estatisticas9c.csv":
dados_estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas9c.csv',
                          index=False)

##### 3. Inclusão de coluna de distancia nas Estatisticas Combinadas 2019

Salvos em ./Dados/DemandaeOferta/estatisticas9c.csv

**Descrição:** Para a análise do dataframe, é interessante ter a variável distância correlacionada às estatísticas combinadas. Para isso, foram identificadas as variáveis correspondentes aos aeroportos de origem e destino de cada voo no arquivo estatisticas9c e correlacionadas às distâncias do arquivo distancias2.

In [ ]:
# Leitura dos arquivos:
estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas9c.csv')

In [ ]:
distancia = pd.read_csv('./Dados/Aerodromos/distancias2.csv')

In [ ]:
# Criando um dicionário com as correspondências entre origem e destino, com as distancias (km):
correspondencias = distancia.set_index(['orig', 'dest'])['dist'].to_dict()

In [ ]:
# Criando a variável km_distancia e mapeando os valores de orig/sg_icao_origem e dest/sg_icao_destino para obter as distâncias correspondentes:
estatisticas['km_distancia'] = estatisticas[['sg_icao_origem', 'sg_icao_destino']].apply(lambda row: correspondencias.get((row['sg_icao_origem'], row['sg_icao_destino'])), axis=1)

In [ ]:
# Visualizando o dataframe:
estatisticas.head()

In [ ]:
# Salvando o arquivo "estatisticas9c.csv" com a variável distância inclusa.
estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas9c.csv', index=False)

##### 4. Inclusão de coluna de Modelo de Aeronave e Número de Assentos nas Estatisticas Combinadas 2019
Salvos em ./Dados/DemandaeOferta/estatisticas9_filtered.csv

**Descrição:** Apesar dos banco de dados de estatísticas combinadas dos voos serem mais adequados para a pesquisa, por representarem a origem e destino dos passageiros no voo - independentemente das escalas realizadas - um grande limitador desses bancos foi a carência de algumas variáveis essenciais para as análises da pesquisa, como o Modelo da Aeronave, Número de Assentos e - principalmente - das Tarifas praticadas para cada voo.

As etapas subsequentes foram realizadas para a inclusão dessas variáveis nos arquivos de estatísticas combinadas, a partir da correlação entre os arquivos de estatisticas básicas e tarifas. As correlações foram realizadas a partir das combinações entre as variáveis-chaves dos arquivos.

In [ ]:
# Leitura dos arquivos:
estatisticasc = pd.read_csv("./Dados/DemandaeOferta/estatisticas9c.csv")
estatisticasb = pd.read_csv("./Dados/DemandaeOferta/estatisticas9.csv")

In [ ]:
# Filtrando o dataframe estatisticas básicas por empresas desejadas:
empresas_desejadas = ["ACN", "PAM", "ASO", "AZU", "PTB", "ABJ"]
estatisticasb_filtrado = estatisticasb[estatisticasb['sg_empresa_icao'].isin(empresas_desejadas)]

In [ ]:
# Filtrando o dataframe estatisticas básicas (filtrado) por aeronaves indesejadas:
aeronaves_indesejadas = ["A20N", "A21N", "A319", "A320", "A321", "A332", "A339", "A359", "B38M", "B737", "B738", "B763", "B77W", "B789", "E195", "E295", "E190"]
condiçao = estatisticasb_filtrado['sg_equipamento_icao'].isin(aeronaves_indesejadas)
estatisticasb_filtrado = estatisticasb_filtrado[~condiçao]

In [ ]:
# Identificando os números de voos com as respectivas aeronaves de interesse:
nr_voos = estatisticasb_filtrado['nr_voo'].unique()
print(nr_voos)

In [ ]:
# Filtrando os números de voos encontrados das estatisticas combinadas:
estatisticasc_filtrado = estatisticasc[estatisticasc['nr_voo'].isin(nr_voos)]

In [ ]:
# Filtrando o dataframe estatisticas combinadas (filtrado) por empresas desejadas:
empresas_desejadas = ['PAM', 'ABJ', 'ACN', 'PTB', 'AZU', 'ASO']
estatisticasc_filtrado = estatisticasc_filtrado[estatisticasc_filtrado['sg_empresa_icao'].isin(empresas_desejadas)]

In [ ]:
estatisticasc_filtrado['sg_empresa_icao'].unique()

In [ ]:
# Incluindo a coluna ICAO de aeronave na estatistica combinada
# Criando um dicionário com as correspondências entre nr_voo/sg_empresa_origem e sg_equipamento_icao
correspondencias = estatisticasb_filtrado.set_index(['nr_voo', 'sg_empresa_icao'])['sg_equipamento_icao'].to_dict()

In [ ]:
# Mapeando os valores de nr_voo/sg_empresa_origem e sg_empresa_icao para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['sg_equipamento_icao'] = estatisticasc_filtrado[['nr_voo', 'sg_empresa_icao']].apply(lambda row: correspondencias.get((row['nr_voo'], row['sg_empresa_icao'])), axis=1)

In [ ]:
# Incluindo a coluna modelo de aeronave na estatistica combinada
# Criando um dicionário com as correspondências entre sg_equipamento_icao e ds_modelo
correspondencias2 = estatisticasb_filtrado.set_index(['sg_equipamento_icao'])['ds_modelo'].to_dict()

In [ ]:
# Mapeando os valores de sg_empresa_icao e ds_modelo para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['ds_modelo'] = estatisticasc_filtrado[['sg_equipamento_icao']].apply(lambda row: correspondencias2.get((row['sg_equipamento_icao'])), axis=1)

In [ ]:
# Incluindo a coluna assentos ofertados na estatistica combinada
# Criando um dicionário com as correspondências entre nr_voo/sg_empresa_icao/sg_equipamento_icao e nr_assentos_ofertados
correspondencias3 = estatisticasb_filtrado.set_index(['nr_voo', 'sg_empresa_icao', 'sg_equipamento_icao'])['nr_assentos_ofertados'].to_dict()

In [ ]:
# Mapeando os valores de nr_voo/sg_empresa_origem e ds_modelo para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['nr_assentos_ofertados'] = estatisticasc_filtrado[['nr_voo', 'sg_empresa_icao', 'sg_equipamento_icao']].apply(lambda row: correspondencias3.get((row['nr_voo'], row['sg_empresa_icao'], row['sg_equipamento_icao'])), axis=1)

In [ ]:
estatisticasc_filtrado.head()

In [ ]:
estatisticasc_filtrado.shape

In [ ]:
# Salvando o arquivo "estatisticas9c_filtered.csv":
estatisticasc_filtrado.to_csv('./Dados/DemandaeOferta/estatisticas9c_filtered.csv', index=False)

##### 5. Inclusão da coluna de Tarifas nas Estatisticas Combinadas 2019
Salvos em ./Dados/DemandaeOferta/estatisticas9c_filtered.csv

**Descrição:** Para incluir a variável Tarifas nos arquivos de estatisticas combinadas, foi necessário criar uma variável de semestre, pois a variável mês do arquivo tarifas não corresponde à variável mês do arquivo estatisticas combinadas. Isso ocorre, porque o mês de compra da passagem não corresponde necessariamente ao mês do voo.

Dessa forma, a criação da variável semestre foi uma tentativa de aproximação das variáveis para correlacionar ambos os bancos de dados e obter a variável tarifa para cada voo no arquivo estatisticas combinadas.

In [ ]:
# Leitura do arquivo:
estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas9c_filtered.csv')

In [ ]:
#Criando a coluna SEMESTRE
estatisticas['SEMESTRE'] = pd.cut(estatisticas['nr_mes_partida_real'], bins=[0, 6, 12], labels=[1, 2])

In [ ]:
#Incluindo a coluna das empresas que comercializam as passagens (sg_comercializa_icao)
empresas = {
    "PAM": "PTB",
    "PTB": "PTB",
    "ACN": "AZU",
    "AZU": "AZU",
    "ABJ": "ABJ"
}
estatisticas['sg_comercializa_icao'] = estatisticas['sg_empresa_icao'].map(empresas)

In [ ]:
estatisticas.head()

###### Adequação do arquivo tarifas9.csv


**Descrição:** Após a criação da variável semestre, foi necessária a criação da variável tarifa ponderada, qua consiste na média ponderada das tarifas praticadas para o mesmo grupo ('ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'), ponderadas pela quantidade de assentos.

Após a criação da variável tarifa ponderada, foram calculados os desvios padrões das tarifas ponderadas.

In [ ]:
# Leitura do arquivo:
tarifas = pd.read_csv('./Dados/Tarifas/tarifas9.csv')

In [ ]:
#Criando a coluna SEMESTRE
tarifas['SEMESTRE'] = pd.cut(tarifas['MES'], bins=[0, 6, 12], labels=[1, 2])

In [ ]:
#Criando a coluna média ponderada das Tarifas
tarifas['produto'] = tarifas['TARIFA'] * tarifas['ASSENTOS']
soma_ponderada = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['produto'].sum()
soma_assentos = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['ASSENTOS'].sum()
TARIFA_PONDERADA = soma_ponderada / soma_assentos

In [ ]:
# Adicionando a variável TARIFA_PONDERADA ao dataframe:
tarifas = tarifas.merge(TARIFA_PONDERADA.rename('TARIFA_PONDERADA'), left_on=['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'], right_index=True)

In [ ]:
# Excluindo a variável criada provisoriamente para o cálculo da média ponderada da tarifa:
tarifas = tarifas.drop(columns='produto')

In [ ]:
tarifas.head()

In [ ]:
# Incluindo a coluna tarifas na estatistica combinada
# Criando um dicionário com as correspondências entre MES/EMPRESA/ORIGEM/DESTINO e TARIFA
correspondencias = tarifas.set_index(['SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['TARIFA_PONDERADA'].to_dict()

In [ ]:
# Mapeando os valores de nr_mes_partida_real/sg_empresa_icao/sg_icao_origem/sg_icao_destino e tarifa para obter os modelos de aeronaves correspondentes
estatisticas['tarifa_ponderada'] = estatisticas[['SEMESTRE', 'sg_comercializa_icao', 'sg_icao_origem', 'sg_icao_destino']].apply(lambda row: correspondencias.get((row['SEMESTRE'], row['sg_comercializa_icao'], row['sg_icao_origem'], row['sg_icao_destino'])), axis=1)

In [ ]:
estatisticas.head()

In [ ]:
#Calculando o Desvio Padrão
tarifas['desvio'] = tarifas['ASSENTOS'] * (tarifas['TARIFA'] - tarifas['TARIFA_PONDERADA'])**2
soma_desvio = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['desvio'].sum()
soma_assentos = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['ASSENTOS'].sum()
desvio_padrao = np.sqrt(soma_desvio / (soma_assentos - 1))

In [ ]:
# Agora, vamos adicionar essa série ao dataframe original
tarifas = tarifas.merge(desvio_padrao.rename('DESVIO_PADRAO'), left_on=['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'], right_index=True)

In [ ]:
tarifas = tarifas.drop(columns='desvio')

In [ ]:
tarifas.head()

In [ ]:
# Incluindo a coluna DESVIO_PADRAO na estatistica combinada
# Criando um dicionário com as correspondências entre SEMESTRE/EMPRESA/ORIGEM/DESTINO/TARIFA_PONDERADA e DESVIO_PADRAO
correspondencias = tarifas.set_index(['SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA_PONDERADA'])['DESVIO_PADRAO'].to_dict()

In [ ]:
# Mapeando os valores de SEMESTRE/sg_empresa_icao/sg_icao_origem/sg_icao_destino/tarifa para obter o desvio_padrao correspondente
estatisticas['desvio_padrao'] = estatisticas[['SEMESTRE', 'sg_comercializa_icao', 'sg_icao_origem', 'sg_icao_destino', 'tarifa_ponderada']].apply(lambda row: correspondencias.get((row['SEMESTRE'], row['sg_comercializa_icao'], row['sg_icao_origem'], row['sg_icao_destino'], row['tarifa_ponderada'])), axis=1)

In [ ]:
estatisticas.head()

In [ ]:
# Salvando o arquivo "estatisticas9c_filtered.csv" com as novas variáveis:
estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas9c_filtered.csv', index=False)

#### Etapa II.II. Organização e Transformação dos Dados - Estatisticas Combinadas 2020

##### 1. Leitura e agrupamento de arquivos de estatísticas combinadas 2020

Salvo em ./Dados/DemandaeOferta/2020_combinada.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de estatisticas combinadas disponibilizados pela ANAC para 2020.

In [ ]:
# Leitura do arquivo:
files = ['combinada2020-01.zip', 'combinada2020-02.zip','combinada2020-03.zip',
         'combinada2020-04.zip', 'combinada2020-05.zip','combinada2020-06.zip',
         'combinada2020-07.zip', 'combinada2020-08.zip','combinada2020-09.zip',
         'combinada2020-10.zip', 'combinada2020-11.zip','combinada2020-12.zip']

In [ ]:
# Criando o dataframe:
df = pd.DataFrame()

In [ ]:
# Concatenando os arquivos no dataframe:
for file in files:
  print(file)
  data = pd.read_csv(f'./Dados/DemandaeOferta/2020_combinada/{file}',
                     encoding='cp1252', delimiter = ';',low_memory=False)
  df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualizando o dataframe:
df.head()

In [ ]:
# Salvando o arquivo "estatisticas_2020_combinadas.csv":
df.to_csv('./Dados/DemandaeOferta/estatisticas_2020_combinadas.csv', index=False)


##### 2. Obtenção das estatísticas combinadas de Demanda e Oferta em 2020

Salvo em /Dados/Estatisticas/estatisticas0c.csv

**Descrição:** Nesta etapa foi realizada a transformação do arquivo estatisticas_2020_combinadas.csv.

In [ ]:
# Leitura do arquivo:
dados_estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas_2020_combinadas.csv',
                                 low_memory=False)

In [ ]:
# Dimensão do arquivo:
dados_estatisticas.shape

In [ ]:
# Padronizando os dados respectivos ao mês:
dados_estatisticas['nr_mes_referencia'].unique()

In [ ]:
dados_estatisticas.columns

In [ ]:
# Filtrando o dataframe com a natureza da etapa de voo = doméstica, serviço referente ao tipo de linha = passageiro e tipo de Voo = regular:
dados_estatisticas = dados_estatisticas[
    (dados_estatisticas['ds_natureza_etapa'] == 'DOMÉSTICA') & \
    (dados_estatisticas['ds_servico_tipo_linha'] == 'PASSAGEIRO') & \
    (dados_estatisticas['ds_grupo_di'] == 'REGULAR')]
# Com isso, foram excluídos os dados que não serão trabalhados nessa pesquisa.

In [ ]:
dados_estatisticas.info()

In [ ]:
# Exclusão dos dados vazios (NA)
dados_estatisticas.isna().sum()

In [ ]:
# Exclusão das variáveis que não serão estudadas na pesquisa:
dados_estatisticas = dados_estatisticas.drop(columns=
    ['id_combinada','id_empresa','sg_empresa_iata','nm_pais','ds_tipo_empresa',
     'nr_singular','id_di','cd_di','ds_di','ds_grupo_di','dt_referencia',
     'nr_ano_referencia','nr_semestre_referencia','nm_semestre_referencia',
     'nr_trimestre_referencia','nm_trimestre_referencia','nr_mes_referencia',
     'nm_mes_referencia','nr_semana_referencia','nm_dia_semana_referencia',
     'nr_dia_referencia','nr_ano_mes_referencia','id_tipo_linha',
     'cd_tipo_linha','ds_tipo_linha','ds_natureza_tipo_linha','ds_servico_tipo_linha',
     'ds_natureza_etapa','nr_ano_partida_real','nr_semestre_partida_real',
     'nm_semestre_partida_real','nr_trimestre_partida_real','nm_trimestre_partida_real',
     'nm_mes_partida_real','nr_ano_mes_partida_real','id_aerodromo_origem',
     'nm_aerodromo_origem','nm_municipio_origem','nm_pais_origem','nm_continente_origem',
     'nr_ano_chegada_real','nr_semestre_chegada_real','nm_semestre_chegada_real',
     'nr_trimestre_chegada_real','nm_trimestre_chegada_real','nm_mes_chegada_real',
     'nr_ano_mes_chegada_real','id_aerodromo_destino','nm_aerodromo_destino',
     'nm_municipio_destino','nm_regiao_destino','nm_pais_destino','nm_continente_destino',
     'nr_escala_destino','id_arquivo','nm_arquivo','nr_linha','dt_sistema']
)

In [ ]:
# Dimensão do dataframe:
dados_estatisticas.shape

In [ ]:
# Salvando o arquivo "estatisticas0c.csv":
dados_estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas0c.csv',
                          index=False)

##### 3. Inclusão de coluna de distancia nas Estatisticas Combinadas 2020

Salvos em ./Dados/DemandaeOferta/estatisticas0c.csv

**Descrição:** Para a análise do dataframe, é interessante ter a variável distância correlacionada às estatísticas combinadas. Para isso, foram identificadas as variáveis correspondentes aos aeroportos de origem e destino de cada voo no arquivo estatisticas0c e correlacionadas às distâncias do arquivo distancias2.

In [ ]:
# Leitura dos arquivos:
estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas0c.csv')

In [ ]:
distancia = pd.read_csv('./Dados/Aerodromos/distancias2.csv')

In [ ]:
# Criando um dicionário com as correspondências entre origem e destino, com as distancias (km):
correspondencias = distancia.set_index(['orig', 'dest'])['dist'].to_dict()

In [ ]:
# Criando a variável km_distancia e mapeando os valores de orig/sg_icao_origem e dest/sg_icao_destino para obter as distâncias correspondentes:
estatisticas['km_distancia'] = estatisticas[['sg_icao_origem', 'sg_icao_destino']].apply(lambda row: correspondencias.get((row['sg_icao_origem'], row['sg_icao_destino'])), axis=1)

In [ ]:
# Visualizando o dataframe:
estatisticas.head()

In [ ]:
# Salvando o arquivo "estatisticas0c.csv" com a variável distância inclusa.
estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas0c.csv', index=False)

###### 4. Inclusão de coluna de Modelo de Aeronave e Número de Assentos nas Estatisticas Combinadas 2020
Salvos em ./Dados/DemandaeOferta/estatisticas0c_filtered.csv

**Descrição:** Apesar dos banco de dados de estatísticas combinadas dos voos serem mais adequados para a pesquisa, por representarem a origem e destino dos passageiros no voo - independentemente das escalas realizadas - um grande limitador desses bancos foi a carência de algumas variáveis essenciais para as análises da pesquisa, como o Modelo da Aeronave, Número de Assentos e - principalmente - das Tarifas praticadas para cada voo.

As etapas subsequentes foram realizadas para a inclusão dessas variáveis nos arquivos de estatísticas combinadas, a partir da correlação entre os arquivos de estatisticas básicas e tarifas. As correlações foram realizadas a partir das combinações entre as variáveis-chaves dos arquivos.

In [ ]:
# Leitura dos arquivos:
estatisticasc = pd.read_csv("./Dados/DemandaeOferta/estatisticas0c.csv")
estatisticasb = pd.read_csv("./Dados/DemandaeOferta/estatisticas0.csv")

In [ ]:
# Filtrando o dataframe estatisticas básicas por empresas desejadas:
empresas_desejadas = ["ACN", "PAM", "ASO", "AZU", "PTB", "ABJ"]
estatisticasb_filtrado = estatisticasb[estatisticasb['sg_empresa_icao'].isin(empresas_desejadas)]

In [ ]:
# Filtrando o dataframe estatisticas básicas (filtrado) por aeronaves indesejadas:
aeronaves_indesejadas = ["A20N", "A21N", "A319", "A320", "A321", "A332", "A339", "A359", "B38M", "B737", "B738", "B763", "B77W", "B789", "E195", "E295", "E190"]
condiçao = estatisticasb_filtrado['sg_equipamento_icao'].isin(aeronaves_indesejadas)
estatisticasb_filtrado = estatisticasb_filtrado[~condiçao]

In [ ]:
# Identificando os números de voos com as respectivas aeronaves de interesse:
nr_voos = estatisticasb_filtrado['nr_voo'].unique()
print(nr_voos)

In [ ]:
# Filtrando os números de voos encontrados das estatisticas combinadas:
estatisticasc_filtrado = estatisticasc[estatisticasc['nr_voo'].isin(nr_voos)]

In [ ]:
# Filtrando o dataframe estatisticas combinadas (filtrado) por empresas desejadas:
empresas_desejadas = ['PAM', 'ABJ', 'ACN', 'PTB', 'AZU', 'ASO']
estatisticasc_filtrado = estatisticasc_filtrado[estatisticasc_filtrado['sg_empresa_icao'].isin(empresas_desejadas)]

In [ ]:
estatisticasc_filtrado['sg_empresa_icao'].unique()

In [ ]:
# Incluindo a coluna ICAO de aeronave na estatistica combinada
# Criando um dicionário com as correspondências entre nr_voo/sg_empresa_origem e sg_equipamento_icao
correspondencias = estatisticasb_filtrado.set_index(['nr_voo', 'sg_empresa_icao'])['sg_equipamento_icao'].to_dict()

In [ ]:
# Mapeando os valores de nr_voo/sg_empresa_origem e sg_empresa_icao para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['sg_equipamento_icao'] = estatisticasc_filtrado[['nr_voo', 'sg_empresa_icao']].apply(lambda row: correspondencias.get((row['nr_voo'], row['sg_empresa_icao'])), axis=1)

In [ ]:
# Incluindo a coluna modelo de aeronave na estatistica combinada
# Criando um dicionário com as correspondências entre sg_equipamento_icao e ds_modelo
correspondencias2 = estatisticasb_filtrado.set_index(['sg_equipamento_icao'])['ds_modelo'].to_dict()

In [ ]:
# Mapeando os valores de sg_empresa_icao e ds_modelo para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['ds_modelo'] = estatisticasc_filtrado[['sg_equipamento_icao']].apply(lambda row: correspondencias2.get((row['sg_equipamento_icao'])), axis=1)

In [ ]:
# Incluindo a coluna assentos ofertados na estatistica combinada
# Criando um dicionário com as correspondências entre nr_voo/sg_empresa_icao/sg_equipamento_icao e nr_assentos_ofertados
correspondencias3 = estatisticasb_filtrado.set_index(['nr_voo', 'sg_empresa_icao', 'sg_equipamento_icao'])['nr_assentos_ofertados'].to_dict()

In [ ]:
# Mapear os valores de nr_voo/sg_empresa_origem e ds_modelo para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['nr_assentos_ofertados'] = estatisticasc_filtrado[['nr_voo', 'sg_empresa_icao', 'sg_equipamento_icao']].apply(lambda row: correspondencias3.get((row['nr_voo'], row['sg_empresa_icao'], row['sg_equipamento_icao'])), axis=1)

In [ ]:
estatisticasc_filtrado.head()

In [ ]:
estatisticasc_filtrado.shape

In [ ]:
# Salvando o arquivo "estatisticas0c_filtered.csv":
estatisticasc_filtrado.to_csv('./Dados/DemandaeOferta/estatisticas0c_filtered.csv', index=False)

##### 5. Inclusão da coluna de Tarifas nas Estatisticas Combinadas 2020
Salvos em ./Dados/DemandaeOferta/estatisticas0c_filtered.csv

**Descrição:** Para incluir a variável Tarifas nos arquivos de estatisticas combinadas, foi necessário criar uma variável de semestre, pois a variável mês do arquivo tarifas não corresponde à variável mês do arquivo estatisticas combinadas. Isso ocorre, porque o mês de compra da passagem não corresponde necessariamente ao mês do voo.

Dessa forma, a criação da variável semestre foi uma tentativa de aproximação das variáveis para correlacionar ambos os bancos de dados e obter a variável tarifa para cada voo no arquivo estatisticas combinadas.

In [ ]:
# Leitura do arquivo:
estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas0c_filtered.csv')

In [ ]:
#Criando a coluna SEMESTRE
estatisticas['SEMESTRE'] = pd.cut(estatisticas['nr_mes_partida_real'], bins=[0, 6, 12], labels=[1, 2])

In [ ]:
#Incluindo a coluna das empresas que comercializam as passagens (sg_comercializa_icao)
empresas = {
    "PAM": "PTB",
    "PTB": "PTB",
    "ACN": "AZU",
    "AZU": "AZU",
    "ABJ": "ABJ"
}
estatisticas['sg_comercializa_icao'] = estatisticas['sg_empresa_icao'].map(empresas)

In [ ]:
estatisticas.head()

###### Adequação do arquivo tarifas0.csv


**Descrição:** Após a criação da variável semestre, foi necessária a criação da variável tarifa ponderada, qua consiste na média ponderada das tarifas praticadas para o mesmo grupo ('ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'), ponderadas pela quantidade de assentos.

Após a criação da variável tarifa ponderada, foram calculados os desvios padrões das tarifas ponderadas.

In [ ]:
# Leitura do arquivo:
tarifas = pd.read_csv('./Dados/Tarifas/tarifas0.csv')

In [ ]:
#Criando a coluna SEMESTRE
tarifas['SEMESTRE'] = pd.cut(tarifas['MES'], bins=[0, 6, 12], labels=[1, 2])

In [ ]:
#Criando a coluna média ponderada das Tarifas
tarifas['produto'] = tarifas['TARIFA'] * tarifas['ASSENTOS']
soma_ponderada = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['produto'].sum()
soma_assentos = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['ASSENTOS'].sum()
TARIFA_PONDERADA = soma_ponderada / soma_assentos

In [ ]:
# Adicionando a variável TARIFA_PONDERADA ao dataframe:
tarifas = tarifas.merge(TARIFA_PONDERADA.rename('TARIFA_PONDERADA'), left_on=['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'], right_index=True)

In [ ]:
# Excluindo a variável criada provisoriamente para o cálculo da média ponderada da tarifa:
tarifas = tarifas.drop(columns='produto')

In [ ]:
tarifas.head()

In [ ]:
# Incluindo a coluna tarifas na estatistica combinada
# Criando um dicionário com as correspondências entre MES/EMPRESA/ORIGEM/DESTINO e TARIFA
correspondencias = tarifas.set_index(['SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['TARIFA_PONDERADA'].to_dict()

In [ ]:
# Mapeando os valores de nr_mes_partida_real/sg_empresa_icao/sg_icao_origem/sg_icao_destino e tarifa para obter os modelos de aeronaves correspondentes
estatisticas['tarifa_ponderada'] = estatisticas[['SEMESTRE', 'sg_comercializa_icao', 'sg_icao_origem', 'sg_icao_destino']].apply(lambda row: correspondencias.get((row['SEMESTRE'], row['sg_comercializa_icao'], row['sg_icao_origem'], row['sg_icao_destino'])), axis=1)

In [ ]:
estatisticas.head()

In [ ]:
#Calculando o Desvio Padrão
tarifas['desvio'] = tarifas['ASSENTOS'] * (tarifas['TARIFA'] - tarifas['TARIFA_PONDERADA'])**2
soma_desvio = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['desvio'].sum()
soma_assentos = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['ASSENTOS'].sum()
desvio_padrao = np.sqrt(soma_desvio / (soma_assentos - 1))

In [ ]:
# Agora, vamos adicionar essa série ao dataframe original
tarifas = tarifas.merge(desvio_padrao.rename('DESVIO_PADRAO'), left_on=['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'], right_index=True)

In [ ]:
tarifas = tarifas.drop(columns='desvio')

In [ ]:
tarifas.head()

In [ ]:
# Incluindo a coluna DESVIO_PADRAO na estatistica combinada
# Criando um dicionário com as correspondências entre SEMESTRE/EMPRESA/ORIGEM/DESTINO/TARIFA_PONDERADA e DESVIO_PADRAO
correspondencias = tarifas.set_index(['SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA_PONDERADA'])['DESVIO_PADRAO'].to_dict()

In [ ]:
# Mapeando os valores de SEMESTRE/sg_empresa_icao/sg_icao_origem/sg_icao_destino/tarifa para obter o desvio_padrao correspondente
estatisticas['desvio_padrao'] = estatisticas[['SEMESTRE', 'sg_comercializa_icao', 'sg_icao_origem', 'sg_icao_destino', 'tarifa_ponderada']].apply(lambda row: correspondencias.get((row['SEMESTRE'], row['sg_comercializa_icao'], row['sg_icao_origem'], row['sg_icao_destino'], row['tarifa_ponderada'])), axis=1)

In [ ]:
estatisticas.head()

In [ ]:
# Salvando o arquivo "estatisticas0c_filtered.csv" com as novas variáveis:
estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas0c_filtered.csv', index=False)

#### Etapa II.III. Organização e Transformação dos Dados - Estatisticas Combinadas 2021

##### 1. Leitura e agrupamento de arquivos de estatísticas combinadas 2021

Salvo em ./Dados/DemandaeOferta/2021_combinada.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de estatisticas combinadas disponibilizados pela ANAC para 2021.

In [ ]:
# Leitura do arquivo:
files = ['combinada2021-01.zip', 'combinada2021-02.zip','combinada2021-03.zip',
         'combinada2021-04.zip', 'combinada2021-05.zip','combinada2021-06.zip',
         'combinada2021-07.zip', 'combinada2021-08.zip','combinada2021-09.zip',
         'combinada2021-10.zip', 'combinada2021-11.zip','combinada2021-12.zip']

In [ ]:
# Criando o dataframe:
df = pd.DataFrame()

In [ ]:
# Concatenando os arquivos no dataframe:
for file in files:
  print(file)
  data = pd.read_csv(f'./Dados/DemandaeOferta/2021_combinada/{file}',
                     encoding='cp1252', delimiter = ';',low_memory=False)
  df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualizando o dataframe:
df.head()

In [ ]:
# Salvando o arquivo "estatisticas_2021_combinadas.csv":
df.to_csv('./Dados/DemandaeOferta/estatisticas_2021_combinadas.csv', index=False)

##### 2. Obtenção das estatísticas combinadas de Demanda e Oferta em 2021

Salvo em /Dados/Estatisticas/estatisticas1c.csv

**Descrição:** Nesta etapa foi realizada a transformação do arquivo estatisticas_2021_combinadas.csv.

In [ ]:
# Leitura do arquivo:
dados_estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas_2021_combinadas.csv',
                                 low_memory=False)

In [ ]:
# Dimensão do arquivo:
dados_estatisticas.shape

In [ ]:
dados_estatisticas['nr_mes_referencia'].unique()

In [ ]:
dados_estatisticas.columns

In [ ]:
# Filtrando o dataframe com a natureza da etapa de voo = doméstica, serviço referente ao tipo de linha = passageiro e tipo de Voo = regular:
dados_estatisticas = dados_estatisticas[
    (dados_estatisticas['ds_natureza_etapa'] == 'DOMÉSTICA') & \
    (dados_estatisticas['ds_servico_tipo_linha'] == 'PASSAGEIRO') & \
    (dados_estatisticas['ds_grupo_di'] == 'REGULAR')]
# Com isso, foram excluídos os dados que não serão trabalhados nessa pesquisa.

In [ ]:
dados_estatisticas.info()

In [ ]:
# Exclusão dos dados vazios do dataframe (NA)
dados_estatisticas.isna().sum()

In [ ]:
# Exclusão das variáveis que não serão estudadas na pesquisa:
dados_estatisticas = dados_estatisticas.drop(columns=
    ['id_combinada','id_empresa','sg_empresa_iata','nm_pais','ds_tipo_empresa',
     'nr_singular','id_di','cd_di','ds_di','ds_grupo_di','dt_referencia',
     'nr_ano_referencia','nr_semestre_referencia','nm_semestre_referencia',
     'nr_trimestre_referencia','nm_trimestre_referencia','nr_mes_referencia',
     'nm_mes_referencia','nr_semana_referencia','nm_dia_semana_referencia',
     'nr_dia_referencia','nr_ano_mes_referencia','id_tipo_linha',
     'cd_tipo_linha','ds_tipo_linha','ds_natureza_tipo_linha','ds_servico_tipo_linha',
     'ds_natureza_etapa','nr_ano_partida_real','nr_semestre_partida_real',
     'nm_semestre_partida_real','nr_trimestre_partida_real','nm_trimestre_partida_real',
     'nm_mes_partida_real','nr_ano_mes_partida_real','id_aerodromo_origem',
     'nm_aerodromo_origem','nm_municipio_origem','nm_pais_origem','nm_continente_origem',
     'nr_ano_chegada_real','nr_semestre_chegada_real','nm_semestre_chegada_real',
     'nr_trimestre_chegada_real','nm_trimestre_chegada_real','nm_mes_chegada_real',
     'nr_ano_mes_chegada_real','id_aerodromo_destino','nm_aerodromo_destino',
     'nm_municipio_destino','nm_regiao_destino','nm_pais_destino','nm_continente_destino',
     'nr_escala_destino','id_arquivo','nm_arquivo','nr_linha','dt_sistema']
)

In [ ]:
# Dimensão do dataframe:
dados_estatisticas.shape

In [ ]:
# Salvando o arquivo "estatisticas1c.csv":
dados_estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas1c.csv',
                          index=False)

##### 3. Inclusão de coluna de distancia nas Estatisticas Combinadas 2021

Salvos em ./Dados/DemandaeOferta/estatisticas1c.csv

**Descrição:** Para a análise do dataframe, é interessante ter a variável distância correlacionada às estatísticas combinadas. Para isso, foram identificadas as variáveis correspondentes aos aeroportos de origem e destino de cada voo no arquivo estatisticas1c e correlacionadas às distâncias do arquivo distancias2.

In [ ]:
# Leitura dos arquivos:
estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas1c.csv')

In [ ]:
distancia = pd.read_csv('./Dados/Aerodromos/distancias2.csv')

In [ ]:
# Criando um dicionário com as correspondências entre origem e destino, com as distancias (km):
correspondencias = distancia.set_index(['orig', 'dest'])['dist'].to_dict()

In [ ]:
# Criando a variável km_distancia e mapeando os valores de orig/sg_icao_origem e dest/sg_icao_destino para obter as distâncias correspondentes:
estatisticas['km_distancia'] = estatisticas[['sg_icao_origem', 'sg_icao_destino']].apply(lambda row: correspondencias.get((row['sg_icao_origem'], row['sg_icao_destino'])), axis=1)

In [ ]:
# Visualizando o dataframe:
estatisticas.head()

In [ ]:
# Salvando o arquivo "estatisticas9c.csv" com a variável distância inclusa.
estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas1c.csv', index=False)

###### 4. Inclusão de coluna de Modelo de Aeronave e Número de Assentos nas Estatisticas Combinadas 2021
Salvos em ./Dados/DemandaeOferta/estatisticas1c_filtered.csv

**Descrição:** Apesar dos banco de dados de estatísticas combinadas dos voos serem mais adequados para a pesquisa, por representarem a origem e destino dos passageiros no voo - independentemente das escalas realizadas - um grande limitador desses bancos foi a carência de algumas variáveis essenciais para as análises da pesquisa, como o Modelo da Aeronave, Número de Assentos e - principalmente - das Tarifas praticadas para cada voo.

As etapas subsequentes foram realizadas para a inclusão dessas variáveis nos arquivos de estatísticas combinadas, a partir da correlação entre os arquivos de estatisticas básicas e tarifas. As correlações foram realizadas a partir das combinações entre as variáveis-chaves dos arquivos.

In [ ]:
# Leitura dos arquivos:
estatisticasc = pd.read_csv("./Dados/DemandaeOferta/estatisticas1c.csv")
estatisticasb = pd.read_csv("./Dados/DemandaeOferta/estatisticas1.csv")

In [ ]:
# Filtrando o dataframe estatisticas básicas por empresas desejadas:
empresas_desejadas = ["ACN", "PAM", "ASO", "AZU", "PTB", "ABJ"]
estatisticasb_filtrado = estatisticasb[estatisticasb['sg_empresa_icao'].isin(empresas_desejadas)]

In [ ]:
# Filtrando o dataframe estatisticas básicas (filtrado) por aeronaves indesejadas:
aeronaves_indesejadas = ["A20N", "A21N", "A319", "A320", "A321", "A332", "A339", "A359", "B38M", "B737", "B738", "B763", "B77W", "B789", "E195", "E295", "E190"]
condiçao = estatisticasb_filtrado['sg_equipamento_icao'].isin(aeronaves_indesejadas)
estatisticasb_filtrado = estatisticasb_filtrado[~condiçao]

In [ ]:
# Identificando os números de voos com as respectivas aeronaves de interesse:
nr_voos = estatisticasb_filtrado['nr_voo'].unique()
print(nr_voos)

In [ ]:
# Filtrando os números de voos encontrados das estatisticas combinadas:
estatisticasc_filtrado = estatisticasc[estatisticasc['nr_voo'].isin(nr_voos)]

In [ ]:
# Filtrando o dataframe estatisticas combinadas (filtrado) por empresas desejadas:
empresas_desejadas = ['PAM', 'ABJ', 'ACN', 'PTB', 'AZU', 'ASO']
estatisticasc_filtrado = estatisticasc_filtrado[estatisticasc_filtrado['sg_empresa_icao'].isin(empresas_desejadas)]

In [ ]:
estatisticasc_filtrado['sg_empresa_icao'].unique()

In [ ]:
# Incluindo a coluna ICAO de aeronave na estatistica combinada
# Criando um dicionário com as correspondências entre nr_voo/sg_empresa_origem e sg_equipamento_icao
correspondencias = estatisticasb_filtrado.set_index(['nr_voo', 'sg_empresa_icao'])['sg_equipamento_icao'].to_dict()

In [ ]:
# Mapeando os valores de nr_voo/sg_empresa_origem e sg_empresa_icao para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['sg_equipamento_icao'] = estatisticasc_filtrado[['nr_voo', 'sg_empresa_icao']].apply(lambda row: correspondencias.get((row['nr_voo'], row['sg_empresa_icao'])), axis=1)

In [ ]:
# Incluindo a coluna modelo de aeronave na estatistica combinada
# Criando um dicionário com as correspondências entre sg_equipamento_icao e ds_modelo
correspondencias2 = estatisticasb_filtrado.set_index(['sg_equipamento_icao'])['ds_modelo'].to_dict()

In [ ]:
# Criando os valores de sg_empresa_icao e ds_modelo para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['ds_modelo'] = estatisticasc_filtrado[['sg_equipamento_icao']].apply(lambda row: correspondencias2.get((row['sg_equipamento_icao'])), axis=1)

In [ ]:
# Incluindo a coluna assentos ofertados na estatistica combinada
# Criando um dicionário com as correspondências entre nr_voo/sg_empresa_icao/sg_equipamento_icao e nr_assentos_ofertados
correspondencias3 = estatisticasb_filtrado.set_index(['nr_voo', 'sg_empresa_icao', 'sg_equipamento_icao'])['nr_assentos_ofertados'].to_dict()

In [ ]:
# Mapeando os valores de nr_voo/sg_empresa_origem e ds_modelo para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['nr_assentos_ofertados'] = estatisticasc_filtrado[['nr_voo', 'sg_empresa_icao', 'sg_equipamento_icao']].apply(lambda row: correspondencias3.get((row['nr_voo'], row['sg_empresa_icao'], row['sg_equipamento_icao'])), axis=1)

In [ ]:
estatisticasc_filtrado.head()

In [ ]:
estatisticasc_filtrado[estatisticasc_filtrado["sg_empresa_icao"] == "ABJ"]

In [ ]:
# Salvando o arquivo "estatisticas1c_filtered.csv":
estatisticasc_filtrado.to_csv('./Dados/DemandaeOferta/estatisticas1c_filtered.csv', index=False)

##### 5. Inclusão da coluna de Tarifas nas Estatisticas Combinadas 2021
Salvos em ./Dados/DemandaeOferta/estatisticas1c_filtered.csv

**Descrição:** Para incluir a variável Tarifas nos arquivos de estatisticas combinadas, foi necessário criar uma variável de semestre, pois a variável mês do arquivo tarifas não corresponde à variável mês do arquivo estatisticas combinadas. Isso ocorre, porque o mês de compra da passagem não corresponde necessariamente ao mês do voo.

Dessa forma, a criação da variável semestre foi uma tentativa de aproximação das variáveis para correlacionar ambos os bancos de dados e obter a variável tarifa para cada voo no arquivo estatisticas combinadas.

In [ ]:
# Leitura do arquivo:
estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas1c_filtered.csv')

In [ ]:
#Criando a coluna SEMESTRE
estatisticas['SEMESTRE'] = pd.cut(estatisticas['nr_mes_partida_real'], bins=[0, 6, 12], labels=[1, 2])

In [ ]:
#Incluindo a coluna das empresas que comercializam as passagens (sg_comercializa_icao)
empresas = {
    "PAM": "PTB",
    "PTB": "PTB",
    "ACN": "AZU",
    "AZU": "AZU",
    "ABJ": "ABJ"
}
estatisticas['sg_comercializa_icao'] = estatisticas['sg_empresa_icao'].map(empresas)

In [ ]:
estatisticas.head()

###### Adequação do arquivo tarifas1.csv


**Descrição:** Após a criação da variável semestre, foi necessária a criação da variável tarifa ponderada, qua consiste na média ponderada das tarifas praticadas para o mesmo grupo ('ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'), ponderadas pela quantidade de assentos.

Após a criação da variável tarifa ponderada, foram calculados os desvios padrões das tarifas ponderadas.

In [ ]:
# Leitura do arquivo:
tarifas = pd.read_csv('./Dados/Tarifas/tarifas1.csv')

In [ ]:
#Criando a coluna SEMESTRE
tarifas['SEMESTRE'] = pd.cut(tarifas['MES'], bins=[0, 6, 12], labels=[1, 2])

In [ ]:
#Criando a coluna média ponderada das Tarifas
tarifas['produto'] = tarifas['TARIFA'] * tarifas['ASSENTOS']
soma_ponderada = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['produto'].sum()
soma_assentos = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['ASSENTOS'].sum()
TARIFA_PONDERADA = soma_ponderada / soma_assentos

In [ ]:
# Adicionando a variável TARIFA_PONDERADA ao dataframe:
tarifas = tarifas.merge(TARIFA_PONDERADA.rename('TARIFA_PONDERADA'), left_on=['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'], right_index=True)

In [ ]:
# Excluindo a variável criada provisoriamente para o cálculo da média ponderada da tarifa:
tarifas = tarifas.drop(columns='produto')

In [ ]:
tarifas.head()

In [ ]:
# Incluindo a coluna tarifas na estatistica combinada
# Criando um dicionário com as correspondências entre MES/EMPRESA/ORIGEM/DESTINO e TARIFA
correspondencias = tarifas.set_index(['SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['TARIFA_PONDERADA'].to_dict()

In [ ]:
# Mapeando os valores de nr_mes_partida_real/sg_empresa_icao/sg_icao_origem/sg_icao_destino e tarifa para obter os modelos de aeronaves correspondentes
estatisticas['tarifa_ponderada'] = estatisticas[['SEMESTRE', 'sg_comercializa_icao', 'sg_icao_origem', 'sg_icao_destino']].apply(lambda row: correspondencias.get((row['SEMESTRE'], row['sg_comercializa_icao'], row['sg_icao_origem'], row['sg_icao_destino'])), axis=1)

In [ ]:
estatisticas.head()

In [ ]:
#Calculando o Desvio Padrão
tarifas['desvio'] = tarifas['ASSENTOS'] * (tarifas['TARIFA'] - tarifas['TARIFA_PONDERADA'])**2
soma_desvio = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['desvio'].sum()
soma_assentos = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['ASSENTOS'].sum()
desvio_padrao = np.sqrt(soma_desvio / (soma_assentos - 1))

In [ ]:
# Agora, vamos adicionar essa série ao dataframe original
tarifas = tarifas.merge(desvio_padrao.rename('DESVIO_PADRAO'), left_on=['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'], right_index=True)

In [ ]:
tarifas = tarifas.drop(columns='desvio')

In [ ]:
tarifas.head()

In [ ]:
# Incluindo a coluna DESVIO_PADRAO na estatistica combinada
# Criando um dicionário com as correspondências entre SEMESTRE/EMPRESA/ORIGEM/DESTINO/TARIFA_PONDERADA e DESVIO_PADRAO
correspondencias = tarifas.set_index(['SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA_PONDERADA'])['DESVIO_PADRAO'].to_dict()

In [ ]:
# Mapeando os valores de SEMESTRE/sg_empresa_icao/sg_icao_origem/sg_icao_destino/tarifa para obter o desvio_padrao correspondente
estatisticas['desvio_padrao'] = estatisticas[['SEMESTRE', 'sg_comercializa_icao', 'sg_icao_origem', 'sg_icao_destino', 'tarifa_ponderada']].apply(lambda row: correspondencias.get((row['SEMESTRE'], row['sg_comercializa_icao'], row['sg_icao_origem'], row['sg_icao_destino'], row['tarifa_ponderada'])), axis=1)

In [ ]:
estatisticas.head()

In [ ]:
# Salvando o arquivo "estatisticas1c_filtered.csv" com as novas variáveis:
estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas1c_filtered.csv', index=False)

#### Etapa II.IV. Organização e Transformação dos Dados - Estatisticas Combinadas 2022

##### 1. Leitura e agrupamento de arquivos de estatísticas combinadas 2022

Salvo em ./Dados/DemandaeOferta/2022_combinada.csv

**Descrição:** Nesta etapa foi realizada a leitura e agrupamento dos arquivos de estatisticas combinadas disponibilizados pela ANAC para 2022.

In [ ]:
# Leitura do arquivo:
files = ['combinada2022-01.zip', 'combinada2022-02.zip','combinada2022-03.zip',
         'combinada2022-04.zip', 'combinada2022-05.zip','combinada2022-06.zip',
         'combinada2022-07.zip', 'combinada2022-08.zip','combinada2022-09.zip',
         'combinada2022-10.zip', 'combinada2022-11.zip','combinada2022-12.zip']

In [ ]:
# Criando o dataframe:
df = pd.DataFrame()

In [ ]:
# Concatenando os arquivos no dataframe:
for file in files:
  print(file)
  data = pd.read_csv(f'./Dados/DemandaeOferta/2022_combinada/{file}',
                     encoding='cp1252', delimiter = ';',low_memory=False)
  df = pd.concat([df, data], axis=0,ignore_index=True)

In [ ]:
# Visualizando o dataframe:
df.head()

In [ ]:
# Salvando o arquivo "estatisticas_2022_combinadas.csv":
df.to_csv('./Dados/DemandaeOferta/estatisticas_2022_combinadas.csv', index=False)

##### 2. Obtenção das estatísticas combinadas de Demanda e Oferta em 2022


Salvo em /Dados/Estatisticas/estatisticas2c.csv

**Descrição:** Nesta etapa foi realizada a transformação do arquivo estatisticas_2022_combinadas.csv.

In [ ]:
# Leitura do arquivo:
dados_estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas_2022_combinadas.csv',
                                 low_memory=False)

In [ ]:
# Dimensão do arquivo:
dados_estatisticas.shape

In [ ]:
# Padronizando os dados respectivos ao mês:
dados_estatisticas['nr_mes_referencia'].unique()

In [ ]:
dados_estatisticas.columns

In [ ]:
# Filtrando o dataframe com a natureza da etapa de voo = doméstica, serviço referente ao tipo de linha = passageiro e tipo de Voo = regular:
dados_estatisticas = dados_estatisticas[
    (dados_estatisticas['ds_natureza_etapa'] == 'DOMÉSTICA') & \
    (dados_estatisticas['ds_servico_tipo_linha'] == 'PASSAGEIRO') & \
    (dados_estatisticas['ds_grupo_di'] == 'REGULAR')]
# Com isso, foram excluídos os dados que não serão trabalhados nessa pesquisa.

In [ ]:
dados_estatisticas.info()

In [ ]:
dados_estatisticas.isna().sum()

In [ ]:
# Exclusão das variáveis que não serão estudadas na pesquisa:
dados_estatisticas = dados_estatisticas.drop(columns=
    ['id_combinada','id_empresa','sg_empresa_iata','nm_pais','ds_tipo_empresa',
     'nr_singular','id_di','cd_di','ds_di','ds_grupo_di','dt_referencia',
     'nr_ano_referencia','nr_semestre_referencia','nm_semestre_referencia',
     'nr_trimestre_referencia','nm_trimestre_referencia','nr_mes_referencia',
     'nm_mes_referencia','nr_semana_referencia','nm_dia_semana_referencia',
     'nr_dia_referencia','nr_ano_mes_referencia','id_tipo_linha',
     'cd_tipo_linha','ds_tipo_linha','ds_natureza_tipo_linha','ds_servico_tipo_linha',
     'ds_natureza_etapa','nr_ano_partida_real','nr_semestre_partida_real',
     'nm_semestre_partida_real','nr_trimestre_partida_real','nm_trimestre_partida_real',
     'nm_mes_partida_real','nr_ano_mes_partida_real','id_aerodromo_origem',
     'nm_aerodromo_origem','nm_municipio_origem','nm_pais_origem','nm_continente_origem',
     'nr_ano_chegada_real','nr_semestre_chegada_real','nm_semestre_chegada_real',
     'nr_trimestre_chegada_real','nm_trimestre_chegada_real','nm_mes_chegada_real',
     'nr_ano_mes_chegada_real','id_aerodromo_destino','nm_aerodromo_destino',
     'nm_municipio_destino','nm_regiao_destino','nm_pais_destino','nm_continente_destino',
     'nr_escala_destino','id_arquivo','nm_arquivo','nr_linha','dt_sistema']
)

In [ ]:
# Dimensão do dataframe:
dados_estatisticas.shape

In [ ]:
# Salvando o arquivo "estatisticas2c.csv":
dados_estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas2c.csv',
                          index=False)

##### 3. Inclusão de coluna de distancia nas Estatisticas Combinadas 2022

Salvos em ./Dados/DemandaeOferta/estatisticas2c.csv

**Descrição:** Para a análise do dataframe, é interessante ter a variável distância correlacionada às estatísticas combinadas. Para isso, foram identificadas as variáveis correspondentes aos aeroportos de origem e destino de cada voo no arquivo estatisticas2c e correlacionadas às distâncias do arquivo distancias2.

In [ ]:
# Leitura dos arquivos:
estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas2c.csv')

In [ ]:
distancia = pd.read_csv('./Dados/Aerodromos/distancias2.csv')

In [ ]:
# Criando um dicionário com as correspondências entre origem e destino, com as distancias (km):
correspondencias = distancia.set_index(['orig', 'dest'])['dist'].to_dict()

In [ ]:
# Criando a variável km_distancia e mapeando os valores de orig/sg_icao_origem e dest/sg_icao_destino para obter as distâncias correspondentes:
estatisticas['km_distancia'] = estatisticas[['sg_icao_origem', 'sg_icao_destino']].apply(lambda row: correspondencias.get((row['sg_icao_origem'], row['sg_icao_destino'])), axis=1)

In [ ]:
# Visualizando o dataframe:
estatisticas.head()

In [ ]:
# Salvando o arquivo "estatisticas2c.csv" com a variável distância inclusa.
estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas2c.csv', index=False)

###### 4. Inclusão de coluna de Modelo de Aeronave e Número de Assentos nas Estatisticas Combinadas 2022
Salvos em ./Dados/DemandaeOferta/estatisticas2c_filtered.csv

**Descrição:** Apesar dos banco de dados de estatísticas combinadas dos voos serem mais adequados para a pesquisa, por representarem a origem e destino dos passageiros no voo - independentemente das escalas realizadas - um grande limitador desses bancos foi a carência de algumas variáveis essenciais para as análises da pesquisa, como o Modelo da Aeronave, Número de Assentos e - principalmente - das Tarifas praticadas para cada voo.

As etapas subsequentes foram realizadas para a inclusão dessas variáveis nos arquivos de estatísticas combinadas, a partir da correlação entre os arquivos de estatisticas básicas e tarifas. As correlações foram realizadas a partir das combinações entre as variáveis-chaves dos arquivos.

In [ ]:
# Leitura dos arquivos:
estatisticasc = pd.read_csv("./Dados/DemandaeOferta/estatisticas2c.csv")
estatisticasb = pd.read_csv("./Dados/DemandaeOferta/estatisticas2.csv")

In [ ]:
# Filtrando o dataframe estatisticas básicas por empresas desejadas:
empresas_desejadas = ["ACN", "PAM", "ASO", "AZU", "PTB", "ABJ"]
estatisticasb_filtrado = estatisticasb[estatisticasb['sg_empresa_icao'].isin(empresas_desejadas)]

In [ ]:
# Filtrando o dataframe estatisticas básicas (filtrado) por aeronaves indesejadas:
aeronaves_indesejadas = ["A20N", "A21N", "A319", "A320", "A321", "A332", "A339", "A359", "B38M", "B737", "B738", "B763", "B77W", "B789", "E195", "E295", "E190"]
condiçao = estatisticasb_filtrado['sg_equipamento_icao'].isin(aeronaves_indesejadas)
estatisticasb_filtrado = estatisticasb_filtrado[~condiçao]

In [ ]:
# Identificando os números de voos com as respectivas aeronaves de interesse:
nr_voos = estatisticasb_filtrado['nr_voo'].unique()
print(nr_voos)

In [ ]:
# Filtrando os números de voos encontrados das estatisticas combinadas:
estatisticasc_filtrado = estatisticasc[estatisticasc['nr_voo'].isin(nr_voos)]

In [ ]:
# Filtrando o dataframe estatisticas combinadas (filtrado) por empresas desejadas:
empresas_desejadas = ['PAM', 'ABJ', 'ACN', 'PTB', 'AZU', 'ASO']
estatisticasc_filtrado = estatisticasc_filtrado[estatisticasc_filtrado['sg_empresa_icao'].isin(empresas_desejadas)]

In [ ]:
estatisticasc_filtrado['sg_empresa_icao'].unique()

In [ ]:
# Incluindo a coluna ICAO de aeronave na estatistica combinada
# Criando um dicionário com as correspondências entre nr_voo/sg_empresa_origem e sg_equipamento_icao
correspondencias = estatisticasb_filtrado.set_index(['nr_voo', 'sg_empresa_icao'])['sg_equipamento_icao'].to_dict()

In [ ]:
# Mapeando os valores de nr_voo/sg_empresa_origem e sg_empresa_icao para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['sg_equipamento_icao'] = estatisticasc_filtrado[['nr_voo', 'sg_empresa_icao']].apply(lambda row: correspondencias.get((row['nr_voo'], row['sg_empresa_icao'])), axis=1)

In [ ]:
# Incluindo a coluna modelo de aeronave na estatistica combinada
# Criando um dicionário com as correspondências entre sg_equipamento_icao e ds_modelo
correspondencias2 = estatisticasb_filtrado.set_index(['sg_equipamento_icao'])['ds_modelo'].to_dict()

In [ ]:
# Mapeando os valores de sg_empresa_icao e ds_modelo para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['ds_modelo'] = estatisticasc_filtrado[['sg_equipamento_icao']].apply(lambda row: correspondencias2.get((row['sg_equipamento_icao'])), axis=1)

In [ ]:
# Incluindo a coluna assentos ofertados na estatistica combinada
# Criando um dicionário com as correspondências entre nr_voo/sg_empresa_icao/sg_equipamento_icao e nr_assentos_ofertados
correspondencias3 = estatisticasb_filtrado.set_index(['nr_voo', 'sg_empresa_icao', 'sg_equipamento_icao'])['nr_assentos_ofertados'].to_dict()

In [ ]:
# Mapeando os valores de nr_voo/sg_empresa_origem e ds_modelo para obter os modelos de aeronaves correspondentes
estatisticasc_filtrado['nr_assentos_ofertados'] = estatisticasc_filtrado[['nr_voo', 'sg_empresa_icao', 'sg_equipamento_icao']].apply(lambda row: correspondencias3.get((row['nr_voo'], row['sg_empresa_icao'], row['sg_equipamento_icao'])), axis=1)

In [ ]:
estatisticasc_filtrado.head()

In [ ]:
estatisticasc_filtrado.shape

In [ ]:
# Salvando o arquivo "estatisticas2c_filtered.csv":
estatisticasc_filtrado.to_csv('./Dados/DemandaeOferta/estatisticas2c_filtered.csv', index=False)

##### 5. Inclusão da coluna de Tarifas nas Estatisticas Combinadas 2022
Salvos em ./Dados/DemandaeOferta/estatisticas2c_filtered.csv

**Descrição:** Para incluir a variável Tarifas nos arquivos de estatisticas combinadas, foi necessário criar uma variável de semestre, pois a variável mês do arquivo tarifas não corresponde à variável mês do arquivo estatisticas combinadas. Isso ocorre, porque o mês de compra da passagem não corresponde necessariamente ao mês do voo.

Dessa forma, a criação da variável semestre foi uma tentativa de aproximação das variáveis para correlacionar ambos os bancos de dados e obter a variável tarifa para cada voo no arquivo estatisticas combinadas.

In [ ]:
# Leitura do arquivo:
estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticas2c_filtered.csv')

In [ ]:
#Criando a coluna SEMESTRE
estatisticas['SEMESTRE'] = pd.cut(estatisticas['nr_mes_partida_real'], bins=[0, 6, 12], labels=[1, 2])

In [ ]:
#Incluindo a coluna das empresas que comercializam as passagens (sg_comercializa_icao)
empresas = {
    "PAM": "PTB",
    "PTB": "PTB",
    "ACN": "AZU",
    "AZU": "AZU",
    "ABJ": "ABJ"
}
estatisticas['sg_comercializa_icao'] = estatisticas['sg_empresa_icao'].map(empresas)

In [ ]:
estatisticas.head()

###### Adequação do arquivo tarifas2.csv


**Descrição:** Após a criação da variável semestre, foi necessária a criação da variável tarifa ponderada, qua consiste na média ponderada das tarifas praticadas para o mesmo grupo ('ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'), ponderadas pela quantidade de assentos.

Após a criação da variável tarifa ponderada, foram calculados os desvios padrões das tarifas ponderadas.

In [ ]:
# Leitura do arquivo:
tarifas = pd.read_csv('./Dados/Tarifas/tarifas2.csv')

In [ ]:
#Criando a coluna SEMESTRE
tarifas['SEMESTRE'] = pd.cut(tarifas['MES'], bins=[0, 6, 12], labels=[1, 2])

In [ ]:
#Criando a coluna média ponderada das Tarifas
tarifas['produto'] = tarifas['TARIFA'] * tarifas['ASSENTOS']
soma_ponderada = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['produto'].sum()
soma_assentos = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['ASSENTOS'].sum()
TARIFA_PONDERADA = soma_ponderada / soma_assentos

In [ ]:
# Adicionando a variável TARIFA_PONDERADA ao dataframe:
tarifas = tarifas.merge(TARIFA_PONDERADA.rename('TARIFA_PONDERADA'), left_on=['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'], right_index=True)

In [ ]:
# Excluindo a variável criada provisoriamente para o cálculo da média ponderada da tarifa:
tarifas = tarifas.drop(columns='produto')

In [ ]:
tarifas.head()

In [ ]:
# Incluindo a coluna tarifas na estatistica combinada
# Criando um dicionário com as correspondências entre MES/EMPRESA/ORIGEM/DESTINO e TARIFA
correspondencias = tarifas.set_index(['SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['TARIFA_PONDERADA'].to_dict()

In [ ]:
# Mapeando os valores de nr_mes_partida_real/sg_empresa_icao/sg_icao_origem/sg_icao_destino e tarifa para obter os modelos de aeronaves correspondentes
estatisticas['tarifa_ponderada'] = estatisticas[['SEMESTRE', 'sg_comercializa_icao', 'sg_icao_origem', 'sg_icao_destino']].apply(lambda row: correspondencias.get((row['SEMESTRE'], row['sg_comercializa_icao'], row['sg_icao_origem'], row['sg_icao_destino'])), axis=1)

In [ ]:
estatisticas.head()

In [ ]:
#Calculando o Desvio Padrão
tarifas['desvio'] = tarifas['ASSENTOS'] * (tarifas['TARIFA'] - tarifas['TARIFA_PONDERADA'])**2
soma_desvio = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['desvio'].sum()
soma_assentos = tarifas.groupby(['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'])['ASSENTOS'].sum()
desvio_padrao = np.sqrt(soma_desvio / (soma_assentos - 1))

In [ ]:
# Agora, vamos adicionar essa série ao dataframe original
tarifas = tarifas.merge(desvio_padrao.rename('DESVIO_PADRAO'), left_on=['ANO', 'SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO'], right_index=True)

In [ ]:
tarifas = tarifas.drop(columns='desvio')

In [ ]:
tarifas.head()

In [ ]:
# Incluindo a coluna DESVIO_PADRAO na estatistica combinada
# Criando um dicionário com as correspondências entre SEMESTRE/EMPRESA/ORIGEM/DESTINO/TARIFA_PONDERADA e DESVIO_PADRAO
correspondencias = tarifas.set_index(['SEMESTRE', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA_PONDERADA'])['DESVIO_PADRAO'].to_dict()

In [ ]:
# Mapeando os valores de SEMESTRE/sg_empresa_icao/sg_icao_origem/sg_icao_destino/tarifa para obter o desvio_padrao correspondente
estatisticas['desvio_padrao'] = estatisticas[['SEMESTRE', 'sg_comercializa_icao', 'sg_icao_origem', 'sg_icao_destino', 'tarifa_ponderada']].apply(lambda row: correspondencias.get((row['SEMESTRE'], row['sg_comercializa_icao'], row['sg_icao_origem'], row['sg_icao_destino'], row['tarifa_ponderada'])), axis=1)

In [ ]:
estatisticas.head()

In [ ]:
# Salvando o arquivo "estatisticas2c_filtered.csv" com as novas variáveis:
estatisticas.to_csv('./Dados/DemandaeOferta/estatisticas2c_filtered.csv', index=False)

## **Passo 4** - Banco de Dados Final

**Descrição:** concatenação final dos bancos de dados tratados neste notebook em um último, que será analisado.

**Atenção:** os arquivos baixados são .CSV, mas no formato Microsoft Excel, *comma separated values*, portanto, não tem o encoding utf-8. Além disso, o Excel em português salva o CSV separado com ponto e vírgula (';').

#### Concatenando os arquivos estatisticas2c_filtered | estatisticas1c_filtered | estatisticas0c_filtered | estatisticas9c_filtered

Salvo em ./Dados/DemandaeOferta/estatisticasc_filtered.csv

In [ ]:
# Leitura dos arquivos:
estatisticas2 = pd.read_csv("./Dados/DemandaeOferta/estatisticas2c_filtered.csv")
estatisticas1 = pd.read_csv("./Dados/DemandaeOferta/estatisticas1c_filtered.csv")
estatisticas0 = pd.read_csv("./Dados/DemandaeOferta/estatisticas0c_filtered.csv")
estatisticas9 = pd.read_csv("./Dados/DemandaeOferta/estatisticas9c_filtered.csv")

In [ ]:
# Eliminação dos dados em branco (NA):
estatisticas2.dropna(subset=["tarifa_ponderada","km_distancia","sg_equipamento_icao"])
estatisticas1.dropna(subset=["tarifa_ponderada","km_distancia","sg_equipamento_icao"])
estatisticas0.dropna(subset=["tarifa_ponderada","km_distancia","sg_equipamento_icao"])
estatisticas9.dropna(subset=["tarifa_ponderada","km_distancia","sg_equipamento_icao"])

In [ ]:
# Incluindo a variável "ano" em cada arquivo:
estatisticas9['ano'] = 2019

In [ ]:
estatisticas0['ano'] = 2020

In [ ]:
estatisticas1['ano'] = 2021

In [ ]:
estatisticas2['ano'] = 2022

In [ ]:
# Concatenando os dataframes em um único arquivo:
estatisticas = pd.concat([estatisticas9, estatisticas0, estatisticas1, estatisticas2])

In [ ]:
# Reordenando a variável "ano":
cols = list(estatisticas.columns)
cols = [cols[-1]] + cols[:-1]
estatisticas = estatisticas[cols]

In [ ]:
# Eliminando as linhas com dados vazios:
estatisticas = estatisticas.dropna(subset=["tarifa_ponderada","km_distancia","sg_equipamento_icao"])
estatisticas.shape

In [ ]:
estatisticas.to_csv('./Dados/DemandaeOferta/estatisticasc_filtered.csv', index=False)

In [ ]:
estatisticas2.to_csv('./Dados/DemandaeOferta/estatisticas2c_filtered.csv', index=False)

In [ ]:
estatisticas1.to_csv('./Dados/DemandaeOferta/estatisticas1c_filtered.csv', index=False)

In [ ]:
estatisticas0.to_csv('./Dados/DemandaeOferta/estatisticas0c_filtered.csv', index=False)

In [ ]:
estatisticas9.to_csv('./Dados/DemandaeOferta/estatisticas9c_filtered.csv', index=False)

## Inclusão de Aeroportos no Banco de Dados ✈

**Descrição:** esse tópico do notebook evidencia alguns dos aeroportos que não estavam contemplados nos dados extraídos da ANAC e que, por tanto, tiveram que ser incluídos para realizar a correlação correta entre o arquivo de distancias e estatísticas.

Foi perceptível a falta desses aeroportos, porque - no momento de correlacionar os arquivos - o código apontou erro e, a partir disso, as variáveis aeroportos foram correlacionadas para identificar os aeroportos que não estavam contemplados no arquivo distancias.

Após identificados os aeroportos, foram identificados a UF e município do aeroporto, o código OACI e a latitude e longitude. Com isso, esses dados foram inclusos no arquivo aeroportos2, viabilizando a inclusão do aeroporto no arquivo distancias2.


##### Acrescentando OACI == 'SBRG' em aeroportos2.csv

In [ ]:
# Leitura dos arquivos:
aeroportos = pd.read_csv('./Dados/Aerodromos/aeroportos2.csv')
distancias = pd.read_csv('./Dados/Aerodromos/distancias2.csv')

In [ ]:
# Criando um dataframe provisório, com os dados do novo aeroporto:
novo_aeroporto = []
novo_aeroporto.append(['SBRG', "Rio Grande", "RS", -32.01667, -52.08333])

In [ ]:
# Foi necessário retirar uma coluna excedente do arquivo 'aeroportos2'
aeroportos = aeroportos.iloc[:,0:5]
aeroportos.head()

In [ ]:
# Criando um dataframe com as mesmas colunas de aeroportos2 e os dados do novo aeroporto
novo_aeroporto = pd.DataFrame(novo_aeroporto, columns=aeroportos.columns)

In [ ]:
# Concatenando o dataframe do novo aeroporto ao de aeroportos2
aeroportos = pd.concat([aeroportos, novo_aeroporto], axis = 0, ignore_index = True)

In [ ]:
# Pesquisando o novo aeroporto no dataframe aeroportos2
aeroportos[aeroportos.OACI == "SBRG"].drop_duplicates()

In [ ]:
# Salvando o arquivo com os novos dados:
aeroportos.to_csv('./Dados/Aerodromos/aeroportos2.csv',index=False)

✅ Agora basta incluir o novo aeroporto no arquivo distancias2, repetindo as Etapas III e IV do Passo 1.

##### Acrescentando OACI == 'SBGS' em aeroportos2.csv

In [ ]:
# Leitura dos arquivos:
aeroportos = pd.read_csv('./Dados/Aerodromos/aeroportos2.csv')
distancias = pd.read_csv('./Dados/Aerodromos/distancias2.csv')

In [ ]:
# Criando um dataframe provisório, com os dados do novo aeroporto:
novo_aeroporto = []
novo_aeroporto.append(['SBGS', "Ponta Grossa", "PR", -25.184444, -50.143889])

In [ ]:
# Foi necessário retirar uma coluna excedente do arquivo 'aeroportos2'
aeroportos = aeroportos.iloc[:,0:5]
aeroportos.head()

In [ ]:
# Criando um dataframe com as mesmas colunas de aeroportos2 e os dados do novo aeroporto
novo_aeroporto = pd.DataFrame(novo_aeroporto, columns=aeroportos.columns)

In [ ]:
# Concatenando o dataframe do novo aeroporto ao de aeroportos2
aeroportos = pd.concat([aeroportos, novo_aeroporto], axis = 0, ignore_index = True)

In [ ]:
# Pesquisando o novo aeroporto no dataframe aeroportos2
aeroportos[aeroportos.OACI == "SBGS"].drop_duplicates()

In [ ]:
# Salvando o arquivo com os novos dados:
aeroportos.to_csv('./Dados/Aerodromos/aeroportos2.csv',index=False)

✅ Agora basta incluir o novo aeroporto no arquivo distancias2, repetindo as Etapas III e IV do Passo 1.



## **Passo 5** - Análise Gráfica dos Dados (2022 / 2021 / 2020 / 2019)

**Descrição:** Nessa etapa foi realizada a análise gráfica do banco de dados trabalhado ao longo da pesquisa.

In [ ]:
# Leitura dos bancos de dados por ano e do concatenado com todos os anos:
estatisticas = pd.read_csv('./Dados/DemandaeOferta/estatisticasc_filtered.csv')
estatisticas2 = pd.read_csv('./Dados/DemandaeOferta/estatisticas2c_filtered.csv')
estatisticas1 = pd.read_csv('./Dados/DemandaeOferta/estatisticas1c_filtered.csv')
estatisticas0 = pd.read_csv('./Dados/DemandaeOferta/estatisticas0c_filtered.csv')
estatisticas9 = pd.read_csv('./Dados/DemandaeOferta/estatisticas9c_filtered.csv')

# Importando o 'stats' para complementar o cálculo da regressão linear:
from scipy import stats

### Análises - Aeronaves: TODAS

**Descrição:** análise dos dados de todas as aeronaves, observando-se a perspectiva geral da amostra.

#### **Análise de Regressão Linear:**  Distância (km_distancia) vs Tarifas (tarifa_ponderada):

In [ ]:
for ano in [2019, 2020, 2021, 2022]:
    df_ano = estatisticas[(estatisticas['ano'] == ano)]
    x = df_ano['km_distancia']
    y = df_ano['tarifa_ponderada']

    # Calculando a regressão linear
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # Criando o gráfico
    plt.figure(figsize=(8, 6))
    sns.regplot(x=x, y=y)

    # Adicionando a equação da reta ao gráfico
    plt.text(0.1, 0.9, f'y = {slope:.2f}x + {intercept:.2f}', transform=plt.gca().transAxes)

    # Adicionando o coeficiente de correlação de Pearson ao gráfico
    plt.text(0.1, 0.8, f'r = {r_value:.2f}', transform=plt.gca().transAxes)

    plt.xlabel('Distancia (km)')
    plt.ylabel('Tarifa')
    plt.title(f'Distância vs Tarifa Ponderada - Ano: {ano} - Aeronave: TODAS')
    plt.show()

#### **Análise de Regressão Linear:**  Distância (km_distancia) vs Tarifas (tarifa_ponderada), por empresa:

In [ ]:
# Personalizando as cores da legenda do gráfico:
cores = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
mapa_cores = {'PTB': cores[0], 'AZU': cores[1], 'ACN': cores[2], 'PAM': cores[3], 'ABJ': cores[4]}

In [ ]:
for ano in [2019, 2020, 2021, 2022]:
    df_ano = estatisticas[(estatisticas['ano'] == ano)]

    # Criando uma nova figura:
    plt.figure(figsize=(10, 8))

    # Obtendo a lista de empresas aéreas:
    empresas = df_ano['sg_empresa_icao'].unique()

    for empresa in empresas:
        # Filtrando o dataframe para a empresa atual
        df_empresa = df_ano[df_ano['sg_empresa_icao'] == empresa]
        x = df_empresa['km_distancia']
        y = df_empresa['tarifa_ponderada']

        # Calcula a regressão linear
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

        # Cria o gráfico de dispersão e a linha de regressão
        sns.regplot(x=x, y=y, color=mapa_cores[empresa], label=f'{empresa}: y = {slope:.2f}x + {intercept:.2f}, r = {r_value:.2f}')

    # Adiciona a legenda
    plt.legend()

    plt.xlabel('Distancia (km)')
    plt.ylabel('Tarifa')
    plt.title(f'Distância(km) vs Tarifa Ponderada - Ano: {ano} - Aeronave: TODAS')
    plt.show()

#### **Gráfico de Dispersão 3D - Análise de Relação:**  Distância(km_distancia) vs Número de Assentos (nr_assentos_ofertados) vs Tarifas (tarifa_ponderada):

In [ ]:
from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

x = estatisticas['nr_assentos_ofertados']
y = estatisticas['km_distancia']
z = estatisticas['tarifa_ponderada']

ax.scatter(x, y, z)

ax.set_xlabel('Número de Assentos Ofertados')
ax.set_ylabel('Distância(km)')
ax.set_zlabel('Tarifa')

plt.title(f'Relação: Nº Assentos Ofertados vs Distância(km) vs Tarifa (2019 a 2022) - Aeronave = TODAS')
plt.show()


#### **Lineplot:**  Tarifa Ponderada Média por Mês (2019 a 2022):

In [ ]:
for ano in [2019, 2020, 2021, 2022]:
    df_ano = estatisticas[(estatisticas['ano'] == ano)]

    # Calculando a média mensal
    media_mensal = df_ano.groupby('nr_mes_partida_real')['tarifa_ponderada'].mean()

    sns.lineplot(x=media_mensal.index, y=media_mensal.values, label=ano)

plt.xticks(np.arange(1, 13), ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.xlabel('Mês')
plt.ylabel('Tarifa Média')
plt.title('Tarifa Ponderada Média por Mês')
plt.legend()
plt.show()

### Análises - Aeronave: C208

**Descrição:** análise dos dados de todas as aeronaves, observando-se a perspectiva geral da amostra.

#### **Análise de Regressão Linear:**  Distância (km_distancia) vs Tarifas (tarifa_ponderada):

In [ ]:
for ano in [2019, 2020, 2021, 2022]:
    df_ano = estatisticas[(estatisticas['ano'] == ano) & (estatisticas['sg_equipamento_icao'] == 'C208')]
    x = df_ano['km_distancia']
    y = df_ano['tarifa_ponderada']

    # Calculando a regressão linear
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # Criando o gráfico
    plt.figure(figsize=(8, 6))
    sns.regplot(x=x, y=y)

    # Adicionando a equação da reta ao gráfico
    plt.text(0.1, 0.9, f'y = {slope:.2f}x + {intercept:.2f}', transform=plt.gca().transAxes)

    # Adicionando o coeficiente de correlação de Pearson ao gráfico
    plt.text(0.1, 0.8, f'r = {r_value:.2f}', transform=plt.gca().transAxes)

    plt.xlabel('Distancia (km)')
    plt.ylabel('Tarifa')
    plt.title(f'Distância vs Tarifa Ponderada - Ano: {ano} - Aeronave: C208')
    plt.show()

#### **Análise de Regressão Linear:**  Distância (km_distancia) vs Tarifas (tarifa_ponderada), por empresa:

In [ ]:
# Personalizando as cores da legenda do gráfico:
cores = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
mapa_cores = {'PTB': cores[0], 'AZU': cores[1], 'ACN': cores[2], 'PAM': cores[3], 'ABJ': cores[4]}

In [ ]:
for ano in [2019, 2020, 2021, 2022]:
    df_ano = estatisticas[(estatisticas['ano'] == ano) & (estatisticas['sg_equipamento_icao'] == 'C208')]

    # Criando uma nova figura:
    plt.figure(figsize=(10, 8))

    # Obtendo a lista de empresas aéreas:
    empresas = df_ano['sg_empresa_icao'].unique()

    for empresa in empresas:
        # Filtrando o dataframe para a empresa atual
        df_empresa = df_ano[df_ano['sg_empresa_icao'] == empresa]
        x = df_empresa['km_distancia']
        y = df_empresa['tarifa_ponderada']

        # Calcula a regressão linear
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

        # Cria o gráfico de dispersão e a linha de regressão
        sns.regplot(x=x, y=y, color=mapa_cores[empresa], label=f'{empresa}: y = {slope:.2f}x + {intercept:.2f}, r = {r_value:.2f}')

    # Adiciona a legenda
    plt.legend()

    plt.xlabel('Distancia (km)')
    plt.ylabel('Tarifa')
    plt.title(f'Distância(km) vs Tarifa Ponderada - Ano: {ano} - Aeronave: C208')
    plt.show()

#### **Gráfico de Dispersão 3D - Análise de Relação:**  Distância(km_distancia) vs Número de Assentos (nr_assentos_ofertados) vs Tarifas (tarifa_ponderada):

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

cores = ['r', 'g', 'b', 'y']

for i, ano in enumerate([2019, 2020, 2021, 2022]):
    df_ano = estatisticas[(estatisticas['ano'] == ano) & (estatisticas['sg_equipamento_icao'] == 'C208')]
    x = df_ano['nr_assentos_ofertados']
    y = df_ano['km_distancia']
    z = df_ano['tarifa_ponderada']

    ax.scatter(x, y, z, color=cores[i], label=ano)

ax.set_xlabel('Número de Assentos Ofertados')
ax.set_ylabel('Distância(km)')
ax.set_zlabel('Tarifa')

plt.title('Relação: Nº Assentos Ofertados vs Distância(km) vs Tarifa - Aeronave = C208')
plt.legend()
plt.show()


#### **Lineplot:**  Tarifa Ponderada Média por Mês (2019 a 2022):

In [ ]:
for ano in [2019, 2020, 2021, 2022]:
    df_ano = estatisticas[(estatisticas['ano'] == ano) & (estatisticas['sg_equipamento_icao'] == 'C208')]

    # Calculando a média mensal
    media_mensal = df_ano.groupby('nr_mes_partida_real')['tarifa_ponderada'].mean()

    sns.lineplot(x=media_mensal.index, y=media_mensal.values, label=ano)

plt.xticks(np.arange(1, 13), ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.xlabel('Mês')
plt.ylabel('Tarifa Média')
plt.title('Tarifa Ponderada Média por Mês - Aeronave: C208')
plt.legend()
plt.show()

### Identificando Outliers (2019 - 2022)

#### Outliers da variável **Tarifa**:

In [ ]:
# Identificando outliers de estatisticas

# Calculando Q1 and Q3:
Q1 = estatisticas['tarifa_ponderada'].quantile(0.25)
Q3 = estatisticas['tarifa_ponderada'].quantile(0.75)

# Faixa interquartil:
IQR = Q3 - Q1

# Encontrando os outliers
outliers = estatisticas[(estatisticas['tarifa_ponderada'] < (Q1 - 1.5 * IQR)) | (estatisticas['tarifa_ponderada'] > (Q3 + 1.5 * IQR))]

outliers.shape

In [ ]:
sns.boxplot(x = 'ano', y = 'tarifa_ponderada', data = estatisticas)
sns.stripplot(x = 'ano', y = 'tarifa_ponderada', data = estatisticas,
              jitter=True,
              marker='o',
              alpha=0.8,
              color="black")
plt.title('Boxplot of Tarifas by Ano')
plt.xticks(rotation=90)  # Rotating x-axis labels for better readability
plt.show()

#### Outliers da variável **Distância(km)**:

In [ ]:
# Identificando outliers de estatisticas

# Calculando Q1 and Q3:
Q1 = estatisticas['km_distancia'].quantile(0.25)
Q3 = estatisticas['km_distancia'].quantile(0.75)

# Faixa interquartil:
IQR = Q3 - Q1

# Encontrando os outliers
outliers = estatisticas[(estatisticas['km_distancia'] < (Q1 - 1.5 * IQR)) | (estatisticas['km_distancia'] > (Q3 + 1.5 * IQR))]

outliers.shape

In [ ]:
sns.boxplot(x = 'ano', y = 'km_distancia', data = estatisticas)
sns.stripplot(x = 'ano', y = 'km_distancia', data = estatisticas,
              jitter=True,
              marker='o',
              alpha=0.8,
              color="black")
plt.title('Boxplot of Tarifas by Ano')
plt.xticks(rotation=90)  # Rotating x-axis labels for better readability
plt.show()